In [198]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [199]:
# Import usual modules
import pandas as pd
import csv
import math
import os
import numpy as np
import openpyxl
import datetime
import re
import string
from bs4 import BeautifulSoup
import requests
import requests

from date_extractor import extract_dates


In [53]:
#URL ='/Users/veesheenyuen/Desktop/DataScience/SAA/Tilastopaja/Asian Games/Tilastopaja - 17th Asian Games - Incheon.html'  # open a locally saved copy of html
URL ='/Users/veesheenyuen/Desktop/DataScience/SAA/Tilastopaja/SEA Games/Tilastopaja - SEA Games - 2023.html'  # open a locally saved copy of html

HTMLFileToBeOpened = open(URL, "r") 
contents = HTMLFileToBeOpened.read() 
soup = BeautifulSoup(contents, 'lxml') 




In [17]:
print(soup.body.prettify()) 


<body onload="document.getElementById('home').className='active'; leader(); SetLoginFocus();">
 <script>
  function load(){
	document.getElementById("siteLoader").style.display="block";
}
 </script>
 <div class="wrapper">
  <div class="top">
   <div class="logo">
    <h1>
     <a ref="https://www.tilastopaja.info">
      TILASTOPAJA
     </a>
    </h1>
   </div>
   <div class="container">
    <ul id="nav">
     <li class="haku">
      <div>
       SEARCH FOR AN ATHLETE
       <form action="https://www.tilastopaja.info/db/ats.php" method="GET">
        <input class="text" name="Name" size="20" style="font-weight: normal;"/>
        <input class="button" type="submit" value="Go!"/>
       </form>
      </div>
     </li>
     <li id="home">
      <a href="https://www.tilastopaja.info/">
       Home
      </a>
     </li>
     <li>
      <a href="#s1">
       Free
      </a>
      <span id="s1">
      </span>
      <ul class="subs">
       <li>
        <a href="https://www.tilastopaja.info/

In [18]:
# Version without corrections to attemps for high jumps etc.  Skips the line entirely if the event does not have an attemps line

# Traverse html with find_all and enumerate
# <TABLE> then <TR> then <TD> then <A> (name) or <B> (heats and stage)

year_pattern='20\d\d'
dash_pattern='\-'

matches = []
lists=[]
counts=[]
values=[]
names=[]
wind=''
stage=''
event=''
gender=''
heat=''
dict_result2=[]  # length 2 dict
row=[]

temp_df = pd.DataFrame(columns=['RANK', 'NAME', 'NATIONALITY', 'RESULT', 'QUALIFICATION', 'COMPETITION', 'YEAR', 'DATE', 'EVENT', 'VENUE', 'GENDER', 'STAGE', 'HEAT', 'WIND', 'DOB', 'REMARKS', 'RX_TIME', 'DICT_RESULT'])

for div in soup.find_all('div', attrs={ "class" : "left"}):  # find appropriate div
    
    info=div.find('h1')  # then find 'h1' to locate event info
        
    text=info.text
    
    list=text.split(',')
    
    result=list[0].split(':')
    
    competition=result[1]
    
    print('EVENT', competition)
    
   # year = re.findall(year_pattern, list[2])

    #print('YEAR', year)
    
  #  dates = extract_dates(text)
    
   # year=dates[0].year

    year_pos = re.search(year_pattern, text)
    
    year = re.findall(year_pattern, text)[0]
    
  #  date = dates[0].strftime("%B")+' '+str(dates[0].day)

    date1 = re.finditer(dash_pattern, text)
    
    
    print('YEAR', year)
    
    for match in date1:
        
        print('MATCH', match.span())
        
        matches.append(match.span())
    
    venue =' '
    
    print('MATCHES', matches[0][1])  # choose first tuple, second element
    
    date = text[matches[0][1]:year_pos.start()]  # extract date
    
    print('DATE 1', date)




    


#h1s = soup.find_all('h1')

#for h1 in h1s:
        
#    text=h1.text
    
#    if 'RESULTS' in text:
    
#        list=text.split(',')
    
#        list1=list[0].split(':')
    
#        print('event', list1)
    
    


for i in soup.find_all('table'): # Look for the table first then find <tr> for each athlete
        
    for element in i.find_all('tr'): # <tr> separates each athlete/section
        
        print('NEW SECTION')
        
       # for td in element.find_all('td'):
    
        counts=[]
        values=[]
        
            
        for count, value in enumerate(element.find_all('td')):
            
            for n in value.find_all('a'):  # extract the name and put into a list
                
                if len(n.text)>5 and '(' not in n.text and '1y' not in n.text and '/' not in n.text:    # take out all the miscellaneous           
                    
                    names.append(n.text) # add 'clean' name to list
                              
            for m in value.find_all('b'):  # extract heats and stage
                                
                print('MMMMM', m.text)
                
                if 'Multievents' in m.text:
                    print('MULTIEVENTS', m.text)
                    
                    sub_event = 'Multievents'
                 
                                       
            if len(lists)==2:  # captures gender
                
               # heat=''
                wind=''
                dict_result=''
                dict_result2=''
              #  stage=''
                
                print('RESET heat, stage HERE')
                
                if '4 x 100m' not in event and '4 x 400m' not in event: # prevent reset of heats and stage unless relay. Relay reset for stage and heat different than other events
                    
                    heat = ''
                    stage = ''
                
                if 'Men' in lists[1] and len(lists[1])==3: # exact match for 'Men'
                    gender = 'Male'
                    
                if 'Women' in lists[1]:
                    gender = 'Female'
                    
                     
      #      if 'Date' in value.text:
      #          date=value.text
                          
            if 'Wind' in value.text:
                wind=value.text
     #           print('WINDDDDD', value.text)
                
            print('TDTD', count, value.text) # extract Heat, Wind Rank, Nationality, DOB, Result, PB/SB
            
            counts.append(count)
            values.append(value.text)
            
                    
        lists=dict(zip(counts,values)) # convert every TD line under each TR section into a dictionary
        
        
        if len(lists)==3 and 'Wind' in lists[2]:  # if index 2 does not contain 'wind' then it's a new stage
            
            heat = lists[1]
            wind = lists[2]  # wind can also appear in list of length 5
            
            if 'Wind' in lists[2]:
                
                wind = lists[2][5:]  # slice out the 'Wind:'
            
            
            dict_result=[]
            
        if len(lists)==3 and 'Wind' not in lists[2]:
            
            if 'Semifinals' in lists[1]:
                
                stage = 'Semifinals'
                
            if 'Heats' in lists[1]:
                
                stage = 'Heats'
                              
            if 'Heat 1' in lists[1]:
                
                heat = 'Heat 1'
                
            if 'Heat 2' in lists[1]:
                
                heat = 'Heat 2'
            
            if 'Heat 3' in lists[1]:
                
                heat = 'Heat 3'
            
            if 'Heat 4' in lists[1]:
                
                heat = 'Heat 4'
            
            
        #    if len(lists[2])>1:
                                
        #        date = lists[2]  # set date only if lists[2] is not null
            
            dict_result = []
                
        if len(lists)==5:  # dict of length=5 contains date, event info. Reset Heat, Stage and Wind when new event ecountered
            
          #  date = lists[2]
            event = lists[1]
            wind = ' '
            stage = ' '
            heat = ' '
            
            print('Reset STAGE, HEAT here')
            
            dict_result = []
            dict_result2 = []
            
            if 'Wind' in lists[4]:
                
                wind = lists[4][5:]  # slice out the 'Wind:'
            
                        
        if len(lists)==11: # when a new athlete is encountered
                        
            dict_result=lists
                        
            
                        
        #    dict_result2 = [] # reset attempts
            
                        
        if len(lists)==2 and 'Men' not in lists[1]: # dict length 2 contains event gender info or attempts info
            dict_result2 = lists
            
                    
        if len(lists)==2 and 'Women' not in lists[1]: # dict length 2 contains event gender info or attempts info
            dict_result2 = lists
        
        
        
        print('EVENT', event, 'GENDER', gender, 'STAGE', stage, 'HEAT', heat, 'WIND', wind, 'DATE', date, 'DICT_RESULT', dict_result, 'ATTEMPTS', dict_result2)
        
        try:
            
            if 'High Jump' in event or 'Pole Vault' in event or 'Long Jump' in event or 'Triple Jump' in event or 'Shot Put' in event or 'Discus Throw' in event or 'Hammer Throw' in event or 'Javelin Throw' in event or 'Decathlon' in event or '4 x 100m' in event or '4 x 400m' in event or 'Heptathlon' in event:

                print('JUMPS HERE')
                print('dict_result2', dict_result2)
                
                if len(dict_result2)>0:   # if list of attempts is present
                    
                    print('LEN(df)', len(temp_df))
                    
                    print('LENGTH > 2')
                          
                    
                    row.append(dict_result[0]) # rank
                    
                    if '4 x 100m' not in event and '4 x 400m' not in event and 'Mixed Relay' not in event:
                        
                        row.append(names[-1])
                    else:
                        row.append(' ')  # name is blank for relays

                 #   row.append(dict_result[1]) # name  (dict_result[1])
                    row.append(dict_result[3]) # nationality
                    row.append(dict_result[5]) # result incl. DNS
                    row.append(dict_result[9]) # Q/q
                    row.append(competition)
                    row.append(year)
                    row.append(date)
                    row.append(event)
                    row.append(venue)
                    row.append(gender)
                    row.append(stage)
                    row.append(heat)
                    row.append(wind)
                    row.append(dict_result[4]) # DOB                    
                    row.append(dict_result[8]+' '+dict_result[2]) # Remarks. PB/SB
                    row.append(dict_result[10]) # Reaction time for 4 x 100m/400m 
                    row.append(dict_result2) # DICT_RESULT
                                        
                else:
                    
                    
                    
                    pass

                    
            
            elif (event and dict_result[3] and dict_result[5]) is not None:
                                    
               #

                row.append(dict_result[0]) # rank
                
                if '4 x 100m' not in event and '4 x 400m' not in event and 'Mixed Relay' not in event:
                        
                    row.append(names[-1])  # choose last name in list as the correct one
                else:
                    row.append(' ')

                
         
               # row.append(dict_result[1]) # name  (dict_result[1])
                row.append(dict_result[3]) # nationality
                row.append(dict_result[5]) # result incl. DNS
                row.append(dict_result[9]) # Q/q
                row.append(competition)
                row.append(year)
                row.append(date)
                row.append(event)
                row.append(venue)
                row.append(gender)
                row.append(stage)
                row.append(heat)
                row.append(wind)
                row.append(dict_result[4]) # DOB                    
                row.append(dict_result[8]+' '+dict_result[2]) # Remarks. PB/SB
                row.append(dict_result[10]) # Reaction time for 4 x 100m/400m 
                row.append(dict_result2) # DICT_RESULT
                
               

            #print(row)
            
        except:
            
            pass
        
        if len(row)>5:
            
            print('ROW', row)
                    
            temp_df.loc[len(temp_df)] = row

        print('LENGTH', len(lists))

        
        dict_result2 = []
        row=[]
        
            
        
        
        
    wind=''
    heat=''
  #  date=''
    
    lists=[]
    dict_result = []
    row=[]
                        
            
                
            
            
            


        
      

EVENT  Southeast Asian Games
YEAR 2023
MATCH (49, 50)
MATCH (53, 54)
MATCHES 50
DATE 1  8 - 12 May 
NEW SECTION
TDTD 0 
 Men's events: 


Please choose men's event
Men  100m
Men  200m
Men  400m
Men  800m
Men  1500m
Men  5000m
Men  10,000m
Men  3000m Steeplechase
Men  110m Hurdles
Men  400m Hurdles
Men  High Jump
Men  Pole Vault
Men  Long Jump
Men  Triple Jump
Men  Shot Put
Men  Discus Throw
Men  Hammer Throw
Men  Javelin Throw
Men  Decathlon
Men  4 x 100m
Men  4 x 400m
Men  4 x 400m Mixed relay
Men Multievents 100m
Men Multievents 400m
Men Multievents 1500m
Men Multievents 110m Hurdles
Men Multievents High Jump
Men Multievents Pole Vault
Men Multievents Long Jump
Men Multievents Shot Put
Men Multievents Discus Throw
Men Multievents Javelin Throw



TDTD 1      
TDTD 2 
 Women's events: 


Please choose women's event
Women  100m
Women  200m
Women  400m
Women  800m
Women  1500m
Women  5000m
Women  10,000m
Women  3000m Steeplechase
Women  100m Hurdles
Women  400m Hurdles
Women  High Jump


LENGTH 11
NEW SECTION
TDTD 0 4
TDTD 1 Sarah DequinanSarah Dequinan ('92)
TDTD 2 
TDTD 3 PHI
TDTD 4 22 Dec 92
TDTD 5 14.77 
TDTD 6 
TDTD 7  
TDTD 8 SB «
TDTD 9 
TDTD 10  
EVENT 100m Hurdles     GENDER Female STAGE   HEAT   WIND   DATE  8 - 12 May  DICT_RESULT {0: '4', 1: "Sarah\xa0DequinanSarah\xa0Dequinan\xa0('92)", 2: '', 3: 'PHI', 4: '22 Dec 92', 5: '14.77 ', 6: '', 7: '\xa0', 8: 'SB «', 9: '', 10: ' '} ATTEMPTS []
ROW ['4', 'Sarah\xa0Dequinan', 'PHI', '14.77 ', '', ' Southeast Asian Games', '2023', ' 8 - 12 May ', '100m Hurdles\xa0\xa0\xa0\xa0', ' ', 'Female', ' ', ' ', ' ', '22 Dec 92', 'SB « ', ' ', []]
LENGTH 11
NEW SECTION
TDTD 0 5
TDTD 1 Norliana KamaruddinNorliana Kamaruddin ('91)
TDTD 2 
TDTD 3 MAS
TDTD 4 26 Mar 91
TDTD 5 15.26 
TDTD 6 
TDTD 7  
TDTD 8 SB «
TDTD 9 
TDTD 10  
EVENT 100m Hurdles     GENDER Female STAGE   HEAT   WIND   DATE  8 - 12 May  DICT_RESULT {0: '5', 1: "Norliana\xa0KamaruddinNorliana\xa0Kamaruddin\xa0('91)", 2: '', 3: 'MAS', 4: '26 Mar 91', 5: '15.26 ', 

In [743]:
temp_df

,RANK,NAME,NATIONALITY,RESULT,QUALIFICATION,COMPETITION,YEAR,DATE,EVENT,VENUE,GENDER,STAGE,HEAT,WIND,DOB,REMARKS,RX_TIME,DICT_RESULT
0,1,Mohamed Zadi,FRA,10.43,,Flanders Cup,2024,20 July,100m,,Male,,,1.1,20 Mar 01,« »,,[]
1,2,Rodrigo Broer,NED,10.53,,Flanders Cup,2024,20 July,100m,,Male,,,1.1,27 Aug 96,« »,,[]
2,3,Andy Velnom,FRA,10.54,,Flanders Cup,2024,20 July,100m,,Male,,,1.1,28 Dec 99,« »,,[]
3,4,Jordan Paquot,BEL,10.55,,Flanders Cup,2024,20 July,100m,,Male,,,1.1,29 Dec 98,« »,,[]
4,5,Theódoros Vrontinós,GRE,10.67,,Flanders Cup,2024,20 July,100m,,Male,,,1.1,5 Jan 01,« »,,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628,6,Gust Van Canneyt,BEL,50.68,,Flanders Cup,2024,20 July,400m,,Male,,,,25 Apr 06,PB »,,[]
629,1,François Scarniet,BEL,49.54,,Flanders Cup,2024,20 July,400m,,Male,,,,13 Aug 93,PB »,,[]
630,2,Stijn Van Nieuwenhove,BEL,49.99,,Flanders Cup,2024,20 July,400m,,Male,,,,18 Jul 00,SB «,,[]
631,3,Dries Van Schuylenbergh,BEL,50.54,,Flanders Cup,2024,20 July,400m,,Male,,,,12 Feb 01,SB «,,[]


In [3]:
# LATEST VERSION
# Check against 2024 Nagase Cup and WA State Championships results

# Traverse html with find_all and enumerate
# <TABLE> then <TR> then <TD> then <A> (name) or <B> (heats and stage)

year_pattern='20\d\d'
dash_pattern='\-'
country_pattern = '[A-Z]{3}'

matches = []
lists=[]
counts=[]
values=[]
names=[]
wind=''
stage=''
event=''
gender=''
heat=''
sub_event=''
division = ''
dict_result2=[]  # length 2 dict
row=[]

temp_df = pd.DataFrame(columns=['RANK', 'NAME', 'NATIONALITY', 'RESULT', 'QUALIFICATION', 'COMPETITION', 'YEAR', 'DATE', 'EVENT', 'VENUE', 'GENDER', 'STAGE', 'HEAT', 'WIND', 'DOB', 'DIVISION', 'REMARKS', 'RX_TIME', 'DICT_RESULT', 'SUB_EVENT'])

for div in soup.find_all('div', attrs={ "class" : "left"}):  # find appropriate div
    
    info=div.find('h1')  # then find 'h1' to locate event info
        
    text=info.text
    
    print('h1', text)
    
    list=text.split(',')
    
    # HOST CITY
    
    host_city = list[1]
    
    print('HOST CITY', host_city)
    
    # COMPETITION
    
    result=list[0].split(':')
    
    competition=result[1]
    
    print('EVENT', competition)
    
    
     # YEAR
        
   # year = re.findall(year_pattern, list[2])

    #print('YEAR', year)
    
  #  dates = extract_dates(text)
    
   # year=dates[0].year

    year_pos = re.search(year_pattern, text)
    
    year = re.findall(year_pattern, text)[0]
    
  #  date = dates[0].strftime("%B")+' '+str(dates[0].day)

    date1 = re.finditer(dash_pattern, text)
    
    
    print('YEAR', year)
    
    # DATE
    
    for match in date1:
        
        print('MATCH', match.span())
        
        matches.append(match.span())
    
    venue =' '
    
    print('MATCHES', matches[0][1])  # choose first tuple, second element
    
    date = text[matches[0][1]:year_pos.start()]  # extract date
    
    print('DATE', date)

    # VENUE
    
    venue = div.find('h2')
    
    venue_text = venue.text
    
    print('VENUE', venue_text[7:])

#h1s = soup.find_all('h1')

#for h1 in h1s:
        
#    text=h1.text
    
#    if 'RESULTS' in text:
    
#        list=text.split(',')
    
#        list1=list[0].split(':')
    
#        print('event', list1)
    
    


for i in soup.find_all('table'): # Look for the table first then find <tr> for each athlete
        
    for element in i.find_all('tr'): # <tr> separates each athlete/section
        
        print('NEW SECTION')
        
       # for td in element.find_all('td'):
    
        counts=[]
        values=[]
        
            
        for count, value in enumerate(element.find_all('td')):
            
            for n in value.find_all('a'):  # extract the name and put into a list
                
                if len(n.text)>5 and '(' not in n.text and '1y' not in n.text and '/' not in n.text:    # take out all the miscellaneous           
                    
                    names.append(n.text) # add 'clean' name to list
                              
            for m in value.find_all('b'):  # extract heats/stage/division e.g. U20/U18 etc.
                                
                print('NEW HEATS/STAGE/DIVISION', m.text)
                
                if 'Multievents' in m.text:
                    print('MULTIEVENTS', m.text)
                    
                    sub_event = 'Multievents'
                
                
                if 'U20' in m.text or 'U18' in m.text or 'U17' in m.text or 'U16' in m.text or 'U15' in m.text or 'U14' in m.text: # From AUS state championships
                    
                    division = m.text
                else:
                    
                    heat = m.text # NEW ADDITION for Nagase Cup
                
                dict_result = ''  # NEW ADDITION for Nagase Cup. Clear dict_result before next heat
                 
                                       
            if len(lists)==2:  # captures gender
                
               # heat=''
                wind=''
                dict_result=''
                dict_result2=''
              #  stage=''
                
                print('RESET heat, stage HERE')
                
                if '4 x 100m' not in event and '4 x 400m' not in event: # prevent reset of heats and stage unless relay. Relay reset for stage and heat different than other events
                    
                    heat = ''
                    stage = ''
                
                if 'Men' in lists[1] and len(lists[1])==3: # exact match for 'Men'
                    gender = 'Male'
                    
                if 'Women' in lists[1]:
                    gender = 'Female'
                    
                     
            if 'Date' in value.text:
                date=value.text
                          
            if 'Wind' in value.text:
                
                string=value.text
                wind=string[5:]  # NEW ADDITION - Nagase Cup - slice out the 'WIND:'
                print('WINDDDDD', wind)
                
     #       print('TDTD', count, value.text) # extract Heat, Wind Rank, Nationality, DOB, Result, PB/SB
            
            counts.append(count)
            values.append(value.text)
            
                    
        lists=dict(zip(counts,values)) # convert every TD line under each TR section into a dictionary
        
        
        if len(lists)==3 and 'Wind' in lists[2]:  # if index 2 does not contain 'wind' then it's a new stage
            
            heat = lists[1]
            wind = lists[2]  # wind can also appear in list of length 5
            
            if 'Wind' in lists[2]:
                
                wind = lists[2][5:]  # slice out the 'Wind:'
            
            
            dict_result=[]
            
        if len(lists)==3 and 'Wind' not in lists[2]:
            
            
            if 'Semifinals' in lists[1]:
                
                stage = 'Semifinals'
                
            if 'Heats' in lists[1]:
                
                stage = 'Heats'
                              
            if 'Heat 1' in lists[1]:
                
                heat = 'Heat 1'
                
            if 'Heat 2' in lists[1]:
                
                heat = 'Heat 2'
            
            if 'Heat 3' in lists[1]:
                
                heat = 'Heat 3'
            
            if 'Heat 4' in lists[1]:
                
                heat = 'Heat 4'
            
            
            if len(lists[2])>1:
                                
                date = lists[2]  # set date only if lists[2] is not null
            
            dict_result = []
                
        if len(lists)==5:  # dict of length=5 contains date, event info. Reset Heat, Stage and Wind when new event ecountered
            
            date = lists[2]
            event = lists[1]
            wind = ' '
            stage = ' '
            heat = ' '
            
            print('Reset STAGE, HEAT here')
            
            dict_result = []
            dict_result2 = []
            
            if 'Wind' in lists[4]:
                
                wind = lists[4][5:]  # slice out the 'Wind:'
                
                print('WINDWIND', wind)
                            
                        
        if len(lists)==11: # when a new athlete is encountered
                        
            dict_result=lists
                        
            
                        
        #    dict_result2 = [] # reset attempts
            
                        
        if len(lists)==2 and 'Men' not in lists[1]: # dict length 2 contains event gender info or attempts info
            dict_result2 = lists
            
                    
        if len(lists)==2 and 'Women' not in lists[1]: # dict length 2 contains event gender info or attempts info
            dict_result2 = lists
        
        
        
        print('EVENT', event, 'GENDER', gender, 'STAGE', stage, 'HEAT', heat, 'WIND', wind, 'DATE', date , 'DIVISION', division, 'DICT_RESULT', dict_result, 'ATTEMPTS', dict_result2)
        
        try:
            
            if 'High Jump' in event or 'Pole Vault' in event or 'Long Jump' in event or 'Triple Jump' in event or 'Shot Put' in event or 'Discus Throw' in event or 'Hammer Throw' in event or 'Javelin Throw' in event or 'Decathlon' in event or '4 x 100m' in event or '4 x 400m' in event or 'Heptathlon' in event:

            
                
                if len(dict_result2)>0:
                    
                #    print('PREVIOUS ROW', temp_df.tail(1))

                
                   # pass
                  #  temp_df.drop(temp_df.tail(1).index,inplace = True)  # drop previous row without the list of attempts
                    
                    
             #       temp_df.loc[row_index,'DICT_RESULTS'] = dict_result2


                    
                    row.append(dict_result[0]) # rank
                    
                    if '4 x 100m' not in event and '4 x 400m' not in event and 'Mixed Relay' not in event:
                        
                        row.append(names[-1])
                    else:
                        row.append(' ')  # name is blank for relays

                    remarks=dict_result[8]+ ' ' + dict_result[2]
                    
                    if 'PB' in remarks:
                        
                        remarks='PB'
                        
                    elif 'SB' in remarks:
                        
                        remarks='SB'
                    
                        
                    row.append(dict_result[3]) # nationality
                    row.append(dict_result[5]) # result incl. DNS
                    row.append(dict_result[9]) # Q/q
                    row.append(competition)
                    row.append(year)
                    row.append(date)
                    row.append(event)
                    row.append(venue)
                    row.append(gender)
                    row.append(stage)
                    row.append(heat)
                    row.append(wind)
                    row.append(dict_result[4]) # DOB
                    row.append(division)
                    row.append(remarks) # Remarks. PB/SB
                    row.append(dict_result[10]) # Reaction time for 4 x 100m/400m 
                    row.append(dict_result2) # DICT_RESULT
                    row.append(sub_event)
                    
                                     
                elif len(dict_result2)==0:  # append row but do not delete previous row
                    
                    row.append(dict_result[0]) # rank
                    
                    if '4 x 100m' not in event and '4 x 400m' not in event and 'Mixed Relay' not in event:
                        
                        row.append(names[-1])
                    else:
                         row.append(' ')  # name is blank for relays

                    remarks=dict_result[8]+ ' ' + dict_result[2]
                    
                    if 'PB' in remarks:
                        
                        remarks='PB'
                        
                    elif 'SB' in remarks:
                        
                        remarks='SB'
                    
                            
                    row.append(dict_result[3]) # nationality
                    row.append(dict_result[5]) # result incl. DNS
                    row.append(dict_result[9]) # Q/q
                    row.append(competition)
                    row.append(year)
                    row.append(date)
                    row.append(event)
                    row.append(venue)
                    row.append(gender)
                    row.append(stage)
                    row.append(heat)
                    row.append(wind)
                    row.append(dict_result[4]) # DOB
                    row.append(division)
                    row.append(remarks) # Remarks. PB/SB
                    row.append(dict_result[10]) # Reaction time for 4 x 100m/400m 
                    row.append(dict_result2) # DICT_RESULT
                    row.append(sub_event)
                  #  pass
                
                                        
                    
            
            elif (event and dict_result[3] and dict_result[5]) is not None:
                                    
               #

                row.append(dict_result[0]) # rank
                
                if '4 x 100m' not in event and '4 x 400m' not in event and 'Mixed Relay' not in event:
                        
                    row.append(names[-1])  # choose last name in list as the correct one
                else:
                    row.append(' ')

                remarks=dict_result[8]+ ' ' + dict_result[2]
                    
                if 'PB' in remarks:
                        
                    remarks='PB'
                        
                elif 'SB' in remarks:
                        
                    remarks='SB'
                    
                
                row.append(dict_result[3]) # nationality
                row.append(dict_result[5]) # result incl. DNS
                row.append(dict_result[9]) # Q/q
                row.append(competition)
                row.append(year)
                row.append(date)
                row.append(event)
                row.append(venue)
                row.append(gender)
                row.append(stage)
                row.append(heat)
                row.append(wind)
                row.append(dict_result[4]) # DOB 
                row.append(division)
                row.append(remarks) # Remarks. PB/SB
                row.append(dict_result[10]) # Reaction time for 4 x 100m/400m 
                row.append(dict_result2) # DICT_RESULT
                row.append(sub_event)
                
                           
        except:
            
            pass
                
        if len(row)>5:
            
            print('ADDED ROW to DF', row)
                        
                  
            temp_df.loc[len(temp_df)] = row
                
            
            print('LENGTH DF', len(temp_df))
        

        print('LENGTH', len(lists))

        
        dict_result2 = []
        row=[]
        
            
        
        
        
    wind=''
    heat=''
    date=''
    
    lists=[]
    dict_result = []
    row=[]
    sub_event=''
                        
            
                
            
            
            


        
      

NameError: name 'soup' is not defined

In [21]:
temp_df.head(50)

,RANK,NAME,NATIONALITY,RESULT,QUALIFICATION,COMPETITION,YEAR,DATE,EVENT,VENUE,GENDER,STAGE,HEAT,WIND,DOB,DIVISION,REMARKS,RX_TIME,DICT_RESULT,SUB_EVENT
0,1,Soraoat Dabbang,THA,10.37,,Southeast Asian Games,2023,12 May,100m,[Arena: Morodok Techo National Stadium],Male,,,,18 Nov 02,,PB,,[],
1,2,Marc Brian Louis,SGP,10.39,,Southeast Asian Games,2023,12 May,100m,[Arena: Morodok Techo National Stadium],Male,,,,7 Aug 02,,PB,,[],
2,3,Khairul Hafiz Jantan,MAS,10.45,,Southeast Asian Games,2023,12 May,100m,[Arena: Morodok Techo National Stadium],Male,,,,22 Jul 98,,« »,,[],
3,3,Muhammad Haiqal Hanafi,MAS,10.45,,Southeast Asian Games,2023,12 May,100m,[Arena: Morodok Techo National Stadium],Male,,,,29 Apr 99,,« »,,[],
4,5,Ngan Ngoc Nghia,VIE,10.46,,Southeast Asian Games,2023,12 May,100m,[Arena: Morodok Techo National Stadium],Male,,,,20 Jul 98,,SB,,[],
5,6,Joshua Chua Hanwei,SGP,10.78,,Southeast Asian Games,2023,12 May,100m,[Arena: Morodok Techo National Stadium],Male,,,,20 Jan 00,,« »,,[],
6,7,Sokong Pen,CAM,10.80,,Southeast Asian Games,2023,12 May,100m,[Arena: Morodok Techo National Stadium],Male,,,,5 Mar 94,,PB,,[],
7,1,Muhammad Haiqal Hanafi,MAS,10.47,,Southeast Asian Games,2023,12 May,100m,[Arena: Morodok Techo National Stadium],Male,Heats,Heat 1,,29 Apr 99,,« »,,[],
8,2,Soraoat Dabbang,THA,10.50,,Southeast Asian Games,2023,12 May,100m,[Arena: Morodok Techo National Stadium],Male,Heats,Heat 1,,18 Nov 02,,SB,,[],
9,3,Wahyu Setiawan,INA,10.66,,Southeast Asian Games,2023,12 May,100m,[Arena: Morodok Techo National Stadium],Male,Heats,Heat 1,,18 Apr 97,,« »,,[],


In [22]:
columns =  temp_df.columns 
    
for col in columns:
    temp_df[col] = temp_df[col].astype(str)
    temp_df[col] = temp_df[col].str.replace('\xa0', ' ', regex=True)
    temp_df[col] = temp_df[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    temp_df[col] = temp_df[col].str.replace('\r', ' ', regex=True)
    temp_df[col] = temp_df[col].str.replace('\n', ' ', regex=True)

    


In [23]:
result_df = temp_df.drop_duplicates(['RANK', 'NAME', 'NATIONALITY', 'RESULT', 'QUALIFICATION', 'COMPETITION', 'YEAR', 'DATE', 'EVENT', 'VENUE', 'GENDER', 'STAGE', 'HEAT', 'WIND', 'DOB', 'REMARKS', 'RX_TIME', 'SUB_EVENT'], keep='last')

In [831]:
result_df.to_csv('nagase.csv', encoding='utf-8')

In [578]:
temp_df[temp_df['NATIONALITY']=='SGP']

,RANK,NAME,NATIONALITY,RESULT,QUALIFICATION,COMPETITION,YEAR,DATE,EVENT,VENUE,GENDER,STAGE,HEAT,WIND,DOB,REMARKS,RX_TIME,DICT_RESULT
187,7,Lucas Le Cong Fun,SGP,14.81,,Commonwealth Youth Games,2023,9 August,110m Hurdles (91.4cm),,Male,,,0.9,11 Jan 06,« »,,[]
216,11,Lucas Le Cong Fun,SGP,1.90,,Commonwealth Youth Games,2023,9 August,High Jump,,Male,,,,11 Jan 06,PB »,,[]
217,12,s Viresh Kumar,SGP,1.85,,Commonwealth Youth Games,2023,9 August,High Jump,,Male,,,,1 Mar 07,PB »,,[]
225,6,Kangli Emery Conrad,SGP,7.12,,Commonwealth Youth Games,2023,8 August,Long Jump,,Male,,,,26 Apr 06,PB »,,[]


In [579]:
temp_df[temp_df['EVENT']=='High Jump']

,RANK,NAME,NATIONALITY,RESULT,QUALIFICATION,COMPETITION,YEAR,DATE,EVENT,VENUE,GENDER,STAGE,HEAT,WIND,DOB,REMARKS,RX_TIME,DICT_RESULT


In [447]:
temp_df.to_csv("2023_SEAG.csv", encoding='utf-8')

In [394]:
asian_games_2014 = temp_df

In [412]:
asian_games_2023 = temp_df

In [425]:
asian_games_2018 = temp_df

In [426]:
asian_games_2014

,RANK,NAME,NATIONALITY,RESULT,QUALIFICATION,COMPETITION,YEAR,DATE,EVENT,VENUE,GENDER,STAGE,HEAT,WIND,DOB,REMARKS,RX_TIME,DICT_RESULT
0,1,Femi Ogunode,QAT,9.93,,17th Asian Games,2014,28 September,100m,Incheon Asiad Main Stadium,Male,,,0.4,15 May 91,PB »,0.184,[]
1,2,Su Bingtian,CHN,10.10,,17th Asian Games,2014,28 September,100m,Incheon Asiad Main Stadium,Male,,,0.4,29 Aug 89,SB «,0.145,[]
2,3,Kei Takase,JPN,10.15,,17th Asian Games,2014,28 September,100m,Incheon Asiad Main Stadium,Male,,,0.4,25 Nov 88,« »,0.152,[]
3,4,Zhang Peimeng,CHN,10.18,,17th Asian Games,2014,28 September,100m,Incheon Asiad Main Stadium,Male,,,0.4,13 Mar 87,« »,0.128,[]
4,5,Reza Ghasemi,IRI,10.25,,17th Asian Games,2014,28 September,100m,Incheon Asiad Main Stadium,Male,,,0.4,24 Jul 87,SB «,0.161,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
930,2,Huang Yu-Ting,TPE,14.36,,17th Asian Games,2014,28 September,100m Hurdles,Incheon Asiad Main Stadium,Female,Heats,Heat 2,-0.5,17 Nov 88,« »,0.239,[]
931,3,Kotchakorn Khamrueangsri,THA,14.52,,17th Asian Games,2014,28 September,100m Hurdles,Incheon Asiad Main Stadium,Female,Heats,Heat 2,-0.5,13 Mar 95,« »,0.222,[]
932,4,Sushmita Singha Roy,IND,14.67,,17th Asian Games,2014,28 September,100m Hurdles,Incheon Asiad Main Stadium,Female,Heats,Heat 2,-0.5,26 Mar 84,« »,0.229,[]
933,5,Yekaterina Voronina,UZB,15.19,,17th Asian Games,2014,28 September,100m Hurdles,Incheon Asiad Main Stadium,Female,Heats,Heat 2,-0.5,16 Feb 92,PB »,0.192,[]


In [427]:
results = pd.concat([asian_games_2014, asian_games_2018], axis=0)

In [428]:
results = pd.concat([results, asian_games_2023], axis=0)

In [429]:
results

,RANK,NAME,NATIONALITY,RESULT,QUALIFICATION,COMPETITION,YEAR,DATE,EVENT,VENUE,GENDER,STAGE,HEAT,WIND,DOB,REMARKS,RX_TIME,DICT_RESULT
0,1,Femi Ogunode,QAT,9.93,,17th Asian Games,2014,28 September,100m,Incheon Asiad Main Stadium,Male,,,0.4,15 May 91,PB »,0.184,[]
1,2,Su Bingtian,CHN,10.10,,17th Asian Games,2014,28 September,100m,Incheon Asiad Main Stadium,Male,,,0.4,29 Aug 89,SB «,0.145,[]
2,3,Kei Takase,JPN,10.15,,17th Asian Games,2014,28 September,100m,Incheon Asiad Main Stadium,Male,,,0.4,25 Nov 88,« »,0.152,[]
3,4,Zhang Peimeng,CHN,10.18,,17th Asian Games,2014,28 September,100m,Incheon Asiad Main Stadium,Male,,,0.4,13 Mar 87,« »,0.128,[]
4,5,Reza Ghasemi,IRI,10.25,,17th Asian Games,2014,28 September,100m,Incheon Asiad Main Stadium,Male,,,0.4,24 Jul 87,SB «,0.161,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
848,3,,SRI,3:30.88,,19th Asian Games,2023,4 October,4 x 400m,HOC Stadium,Female,,,,,,,"{0: '\xa0', 1: ' Nadeesha Ramanayake, M.H.Jaye..."
849,4,,VIE,3:31.61,,19th Asian Games,2023,4 October,4 x 400m,HOC Stadium,Female,,,,,,,"{0: '\xa0', 1: ' Nguyen Thi Ngoc, Hoang Thi Mi..."
850,5,,PHI,3:40.78,,19th Asian Games,2023,4 October,4 x 400m,HOC Stadium,Female,,,,,,,"{0: '\xa0', 1: ' Robyn Brown, Angel Frank, Mau..."
851,6,,THA,3:50.44,,19th Asian Games,2023,4 October,4 x 400m,HOC Stadium,Female,,,,,,,"{0: '\xa0', 1: ' Sukanya Janchaona, Benny Nont..."


In [430]:
results.to_csv('results.csv', encoding='utf-8')

# Automate Login

In [11]:
# Automate login and scraping

# the URL of the login page
login_url = "https://www.tilastopaja.info/login.php"

# the payload with your login credentials
payload = {
    "user": "SAA",
    "password": "Alexisthe12.",
}

# send the POST request to login
response = requests.post(login_url, data=payload)


# if the request went Ok, you should get a 200 status
print(f"Status code: {response.status_code}")

# parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.text, "html.parser")

# find the page title
page_title = soup.title.string

# print the result page title
print(f"Page title: {page_title}")

Status code: 200
Page title: Tilastopaja Login


# Open Directory Files and Process

In [621]:
# Define scraping function (OLD version. Produces duplicates in results with list of attempts)

def scraper(soup, file):
    
        # Traverse html with find_all and enumerate
    # <TABLE> then <TR> then <TD> then <A> (name) or <B> (heats and stage)

    year_pattern='20\d\d'
    
    dash_pattern='\-'

    lists=[]
    counts=[]
    values=[]
    names=[]
    wind=''
    stage=''
    event=''
    gender=''
    heat=''
    dict_result2=[]  # length 2 dict
    row=[]

    temp_df = pd.DataFrame(columns=['RANK', 'NAME', 'COUNTRY', 'RESULT', 'QUALIFICATION', 'COMPETITION', 'YEAR', 'DATE', 'EVENT', 'VENUE', 'GENDER', 'STAGE', 'HEAT', 'WIND', 'DOB', 'REMARKS', 'RX_TIME', 'DICT_RESULTS', 'SOURCE', 'REGION'])

    for div in soup.find_all('div', attrs={ "class" : "left"}):  # find appropriate div

        info=div.find('h1')  # then find 'h1' to locate event info
        
        text=info.text
    
        list=text.split(',')
    
        result=list[0].split(':')

        competition=result[1]

        print('EVENT', competition)

       # year = re.findall(year_pattern, list[2])

        #print('YEAR', year)

      #  dates = extract_dates(text)

       # year=dates[0].year

##        year_pos = re.search(year_pattern, text)

        year = re.findall(year_pattern, text)[0]

      #  date = dates[0].strftime("%B")+' '+str(dates[0].day)

##        date_pos = re.search(dash_pattern, text)


  #      print('YEAR', year)

  #      for match in date1:

  #          print('MATCH', match.span())

  #          matches.append(match.span())

        venue =' '
        
   #     if len(matches)>1:

    #        print('MATCHES', matches[0][1])  # choose first tuple, second element            
            
    #        date = text[matches[0][1]:year_pos.start()]  # extract date
            
    #    else:
            
    #        date = text[matches[0].end():year_pos.start()]

   ##     date = text[date_pos.end():year_pos.start()]

        date = year
    
        print('DATE', date)



    for i in soup.find_all('table'): # Look for the table first then find <tr> for each athlete

        for element in i.find_all('tr'): # <tr> separates each athlete/section

            print('NEW SECTION')

           # for td in element.find_all('td'):

            counts=[]
            values=[]


            for count, value in enumerate(element.find_all('td')):

                for n in value.find_all('a'):  # extract the name and put into a list

                    if len(n.text)>5 and '(' not in n.text and '1y' not in n.text and '/' not in n.text:    # take out all the miscellaneous           

                        names.append(n.text) # add 'clean' name to list

                for m in value.find_all('b'):  # extract heats and stage

                    print('MMMMM', m.text)


                if len(lists)==2:  # captures gender

                   # heat=''
                    wind=''
                    dict_result=''
                    dict_result2=''
                  #  stage=''

                    print('RESET heat, stage HERE')

                    if '4 x 100m' not in event and '4 x 400m' not in event: # prevent reset of heats and stage unless relay. Relay reset for stage and heat different than other events

                        heat = ''
                        stage = ''

                    if 'Men' in lists[1] and len(lists[1])==3: # exact match for 'Men'
                        gender = 'Male'

                    if 'Women' in lists[1]:
                        gender = 'Female'


                if 'Date' in value.text:
                    date=value.text

                if 'Wind' in value.text:
                    wind=value.text
         #           print('WINDDDDD', value.text)

                print('TDTD', count, value.text) # extract Heat, Wind Rank, Nationality, DOB, Result, PB/SB

                counts.append(count)
                values.append(value.text)


            lists=dict(zip(counts,values)) # convert every TD line under each TR section into a dictionary


            if len(lists)==3 and 'Wind' in lists[2]:  # if index 2 does not contain 'wind' then it's a new stage

                heat = lists[1]
                wind = lists[2]  # wind can also appear in list of length 5

                if 'Wind' in lists[2]:

                    wind = lists[2][5:]  # slice out the 'Wind:'


                dict_result=[]

            if len(lists)==3 and 'Wind' not in lists[2]:

                if 'Semifinals' in lists[1]:

                    stage = 'Semifinals'

                if 'Heats' in lists[1]:

                    stage = 'Heats'

                if 'Heat 1' in lists[1]:

                    heat = 'Heat 1'

                if 'Heat 2' in lists[1]:

                    heat = 'Heat 2'

                if 'Heat 3' in lists[1]:

                    heat = 'Heat 3'

                if 'Heat 4' in lists[1]:

                    heat = 'Heat 4'


                if len(lists[2])>1:
                    
                    pass

                  #  date = lists[2]  # set date only if lists[2] is not null

                dict_result = []

            if len(lists)==5:  # dict of length=5 contains date, event info. Reset Heat, Stage and Wind when new event ecountered

              #  date = lists[2]
                event = lists[1]
                wind = ' '
                stage = ' '
                heat = ' '

                print('Reset STAGE, HEAT here')

                dict_result = []
                dict_result2 = []

                if 'Wind' in lists[4]:

                    wind = lists[4][5:]  # slice out the 'Wind:'


            if len(lists)==11: # when a new athlete is encountered

                dict_result=lists



            #    dict_result2 = [] # reset attempts


            if len(lists)==2 and 'Men' not in lists[1]: # dict length 2 contains event gender info or attempts info
                dict_result2 = lists


            if len(lists)==2 and 'Women' not in lists[1]: # dict length 2 contains event gender info or attempts info
                dict_result2 = lists



        #    print('EVENT', event, 'GENDER', gender, 'STAGE', stage, 'HEAT', heat, 'WIND', wind, 'DATE', date, 'DICT_RESULT', dict_result, 'ATTEMPTS', dict_result2)

            try:

                if 'High Jump' in event or 'Pole Vault' in event or 'Long Jump' in event or 'Triple Jump' in event or 'Shot Put' in event or 'Discus Throw' in event or 'Hammer Throw' in event or 'Javelin Throw' in event or 'Decathlon' in event or '4 x 100m' in event or '4 x 400m' in event or 'Heptathlon' in event:
                    print('dict_result2', dict_result2)

                    
                ###################    
                #    if len(dict_result2)>0:


                 #       row.append(dict_result[0]) # rank

                 #       if '4 x 100m' not in event and '4 x 400m' not in event and 'Mixed Relay' not in event:

                 #           row.append(names[-1])
                 #       else:
                 #           row.append(' ')

                     #   row.append(dict_result[1]) # name  (dict_result[1])
                 #       row.append(dict_result[3]) # nationality
                 #       row.append(dict_result[5]) # result incl. DNS
                 #       row.append(dict_result[9]) # Q/q
                 #       row.append(competition)
                 #       row.append(year)
                 #       row.append(date)
                 #       row.append(event)
                 #       row.append(venue)
                 #       row.append(gender)
                 #       row.append(stage)
                 #       row.append(heat)
                 #       row.append(wind)
                 #       row.append(dict_result[4]) # DOB                    
                 #       row.append(dict_result[8]+' '+dict_result[2]) # Remarks. PB/SB
                 #       row.append(dict_result[10]) # Reaction time for 4 x 100m/400m 
                 #       row.append(dict_result2) # DICT_RESULT
                 #       row.append(file)
                 #       row.append('International')

                  #  else:

                 #       pass
                    
                ##############  
                
                
                    if temp_df['DICT_RESULT'].iloc[:-1] is None: # if last element in previous row does not contain a list of attempts
                
                        temp_df.drop(temp_df.tail(1).index,inplace = True)  # drop previous row without the list of attempts

                    
                   # temp_df.drop(temp_df.tail(1).index,inplace = True)  # drop previous row without the list of attempts

                    
                    row.append(dict_result[0]) # rank
                    
                    if '4 x 100m' not in event and '4 x 400m' not in event and 'Mixed Relay' not in event:
                        
                        row.append(names[-1])
                    else:
                        row.append(' ')  # name is blank for relays

                 #   row.append(dict_result[1]) # name  (dict_result[1])
                    row.append(dict_result[3]) # nationality
                    row.append(dict_result[5]) # result incl. DNS
                    row.append(dict_result[9]) # Q/q
                    row.append(competition)
                    row.append(year)
                    row.append(date)
                    row.append(event)
                    row.append(venue)
                    row.append(gender)
                    row.append(stage)
                    row.append(heat)
                    row.append(wind)
                    row.append(dict_result[4]) # DOB                    
                    row.append(dict_result[8]+' '+dict_result[2]) # Remarks. PB/SB
                    row.append(dict_result[10]) # Reaction time for 4 x 100m/400m 
                    row.append(dict_result2) # DICT_RESULT
                    row.append(file)
                    row.append('International')

                    
                    #print('len(temp_df)', len(temp_df))
                    #print('LAST ROW', temp_df.tail(1))
                                     
                elif len(dict_result2)==0:  # append row but do not delete previous row
                    
                    row.append(dict_result[0]) # rank
                    
                    if '4 x 100m' not in event and '4 x 400m' not in event and 'Mixed Relay' not in event:
                        
                        row.append(names[-1])
                    else:
                         row.append(' ')  # name is blank for relays

                    row.append(dict_result[3]) # nationality
                    row.append(dict_result[5]) # result incl. DNS
                    row.append(dict_result[9]) # Q/q
                    row.append(competition)
                    row.append(year)
                    row.append(date)
                    row.append(event)
                    row.append(venue)
                    row.append(gender)
                    row.append(stage)
                    row.append(heat)
                    row.append(wind)
                    row.append(dict_result[4]) # DOB                    
                    row.append(dict_result[8]+' '+dict_result[2]) # Remarks. PB/SB
                    row.append(dict_result[10]) # Reaction time for 4 x 100m/400m 
                    row.append(dict_result2) # DICT_RESULT
                    row.append(file)
                    row.append('International')

                  #  pass
                


                elif (event and dict_result[3] and dict_result[5]) is not None:

                   #

                    row.append(dict_result[0]) # rank

                    if '4 x 100m' not in event and '4 x 400m' not in event and 'Mixed Relay' not in event:

                        row.append(names[-1])  # choose last name in list as the correct one
                    else:
                        row.append(' ')



                   # row.append(dict_result[1]) # name  (dict_result[1])
                    row.append(dict_result[3]) # nationality
                    row.append(dict_result[5]) # result incl. DNS
                    row.append(dict_result[9]) # Q/q
                    row.append(competition)
                    row.append(year)
                    row.append(date)
                    row.append(event)
                    row.append(venue)
                    row.append(gender)
                    row.append(stage)
                    row.append(heat)
                    row.append(wind)
                    row.append(dict_result[4]) # DOB                    
                    row.append(dict_result[8]+' '+dict_result[2]) # Remarks. PB/SB
                    row.append(dict_result[10]) # Reaction time for 4 x 100m/400m 
                    row.append(dict_result2) # DICT_RESULT
                    row.append(file)
                    row.append('International')



                #print(row)

            except:

                pass

            if len(row)>5:



                temp_df.loc[len(temp_df)] = row

            print('LENGTH', len(lists))


            dict_result2 = []
            row=[]

        wind=''
        heat=''
   #     date=''

        lists=[]
        dict_result = []
        row=[]
                                  
    return temp_df   
                
            
            
            




In [729]:
# 2ND NEWEST VERSION

def scraper_new(soup, file):

    year_pattern='20\d\d'
    dash_pattern='\-'

    matches = []
    lists=[]
    counts=[]
    values=[]
    names=[]
    wind=''
    stage=''
    event=''
    gender=''
    heat=''
    dict_result2=[]  # length 2 dict
    row=[]

    temp_df = pd.DataFrame(columns=['RANK', 'NAME', 'NATIONALITY', 'RESULT', 'QUALIFICATION', 'COMPETITION', 'YEAR', 'DATE', 'EVENT', 'VENUE', 'GENDER', 'STAGE', 'HEAT', 'WIND', 'DOB', 'REMARKS', 'RX_TIME', 'DICT_RESULT', 'SOURCE', 'REGION'])

    for div in soup.find_all('div', attrs={ "class" : "left"}):  # find appropriate div

        info=div.find('h1')  # then find 'h1' to locate event info

        text=info.text

        list=text.split(',')

        result=list[0].split(':')

        competition=result[1]

        print('EVENT', competition)

       # year = re.findall(year_pattern, list[2])

        #print('YEAR', year)

      #  dates = extract_dates(text)

       # year=dates[0].year

        year_pos = re.search(year_pattern, text)

        year = re.findall(year_pattern, text)[0]

      #  date = dates[0].strftime("%B")+' '+str(dates[0].day)

        date1 = re.finditer(dash_pattern, text)


        print('YEAR', year)

        for match in date1:

            print('MATCH', match.span())

            matches.append(match.span())

        venue =' '

        print('MATCHES', matches[0][1])  # choose first tuple, second element

        date = text[matches[0][1]:year_pos.start()]  # extract date

        print('DATE', date)







    #h1s = soup.find_all('h1')

    #for h1 in h1s:

    #    text=h1.text

    #    if 'RESULTS' in text:

    #        list=text.split(',')

    #        list1=list[0].split(':')

    #        print('event', list1)




    for i in soup.find_all('table'): # Look for the table first then find <tr> for each athlete

        for element in i.find_all('tr'): # <tr> separates each athlete/section

       #     print('NEW SECTION')

           # for td in element.find_all('td'):

            counts=[]
            values=[]


            for count, value in enumerate(element.find_all('td')):

                for n in value.find_all('a'):  # extract the name and put into a list

                    if len(n.text)>5 and '(' not in n.text and '1y' not in n.text and '/' not in n.text:    # take out all the miscellaneous           

                        names.append(n.text) # add 'clean' name to list

                for m in value.find_all('b'):  # extract heats and stage

          #          print('MMMMM', m.text)


                if len(lists)==2:  # captures gender

                   # heat=''
                    wind=''
                    dict_result=''
                    dict_result2=''
                  #  stage=''

                    print('RESET heat, stage HERE')

                    if '4 x 100m' not in event and '4 x 400m' not in event: # prevent reset of heats and stage unless relay. Relay reset for stage and heat different than other events

                        heat = ''
                        stage = ''

                    if 'Men' in lists[1] and len(lists[1])==3: # exact match for 'Men'
                        gender = 'Male'

                    if 'Women' in lists[1]:
                        gender = 'Female'


                if 'Date' in value.text:
                    date=value.text

                if 'Wind' in value.text:
                    wind=value.text
         #           print('WINDDDDD', value.text)

         #       print('TDTD', count, value.text) # extract Heat, Wind Rank, Nationality, DOB, Result, PB/SB

                counts.append(count)
                values.append(value.text)


            lists=dict(zip(counts,values)) # convert every TD line under each TR section into a dictionary


            if len(lists)==3 and 'Wind' in lists[2]:  # if index 2 does not contain 'wind' then it's a new stage

                heat = lists[1]
                wind = lists[2]  # wind can also appear in list of length 5

                if 'Wind' in lists[2]:

                    wind = lists[2][5:]  # slice out the 'Wind:'


                dict_result=[]

            if len(lists)==3 and 'Wind' not in lists[2]:

                if 'Semifinals' in lists[1]:

                    stage = 'Semifinals'

                if 'Heats' in lists[1]:

                    stage = 'Heats'

                if 'Heat 1' in lists[1]:

                    heat = 'Heat 1'

                if 'Heat 2' in lists[1]:

                    heat = 'Heat 2'

                if 'Heat 3' in lists[1]:

                    heat = 'Heat 3'

                if 'Heat 4' in lists[1]:

                    heat = 'Heat 4'


                if len(lists[2])>1:

                    date = lists[2]  # set date only if lists[2] is not null

                dict_result = []

            if len(lists)==5:  # dict of length=5 contains date, event info. Reset Heat, Stage and Wind when new event ecountered

                date = lists[2]
                event = lists[1]
                wind = ' '
                stage = ' '
                heat = ' '

        #        print('Reset STAGE, HEAT here')

                dict_result = []
                dict_result2 = []

                if 'Wind' in lists[4]:

                    wind = lists[4][5:]  # slice out the 'Wind:'


            if len(lists)==11: # when a new athlete is encountered

                dict_result=lists



            #    dict_result2 = [] # reset attempts


            if len(lists)==2 and 'Men' not in lists[1]: # dict length 2 contains event gender info or attempts info
                dict_result2 = lists


            if len(lists)==2 and 'Women' not in lists[1]: # dict length 2 contains event gender info or attempts info
                dict_result2 = lists



           # print('EVENT', event, 'GENDER', gender, 'STAGE', stage, 'HEAT', heat, 'WIND', wind, 'DATE', date, 'DICT_RESULT', dict_result, 'ATTEMPTS', dict_result2)

            try:

                if 'High Jump' in event or 'Pole Vault' in event or 'Long Jump' in event or 'Triple Jump' in event or 'Shot Put' in event or 'Discus Throw' in event or 'Hammer Throw' in event or 'Javelin Throw' in event or 'Decathlon' in event or '4 x 100m' in event or '4 x 400m' in event or 'Heptathlon' in event:

                    if len(dict_result2)>0:  # If list of attemps is found

                    #    print('dict_result2', dict_result2)

                    #    print('LEN(df)', len(temp_df))

                

                        row.append(dict_result[0]) # rank

                        if '4 x 100m' not in event and '4 x 400m' not in event and 'Mixed Relay' not in event:

                            row.append(names[-1])  # name
                        else:
                            row.append(' ')  # name is blank for relays

                        row.append(dict_result[3]) # nationality
                        row.append(dict_result[5]) # result incl. DNS
                        row.append(dict_result[9]) # Q/q
                        row.append(competition)
                        row.append(year)
                        row.append(date)
                        row.append(event)
                        row.append(venue)
                        row.append(gender)
                        row.append(stage)
                        row.append(heat)
                        row.append(wind)
                        row.append(dict_result[4]) # DOB                    
                        row.append(dict_result[8]+' '+dict_result[2]) # Remarks. PB/SB
                        row.append(dict_result[10]) # Reaction time for 4 x 100m/400m 
                        row.append(dict_result2) # DICT_RESULT
                        row.append(file)
                        row.append('International')


                  
                    elif len(dict_result2)==0:  # append row but do not delete previous row

                        row.append(dict_result[0]) # rank

                        if '4 x 100m' not in event and '4 x 400m' not in event and 'Mixed Relay' not in event:

                            row.append(names[-1])
                        else:
                             row.append(' ')  # name is blank for relays

                        row.append(dict_result[3]) # nationality
                        row.append(dict_result[5]) # result incl. DNS
                        row.append(dict_result[9]) # Q/q
                        row.append(competition)
                        row.append(year)
                        row.append(date)
                        row.append(event)
                        row.append(venue)
                        row.append(gender)
                        row.append(stage)
                        row.append(heat)
                        row.append(wind)
                        row.append(dict_result[4]) # DOB                    
                        row.append(dict_result[8]+' '+dict_result[2]) # Remarks. PB/SB
                        row.append(dict_result[10]) # Reaction time for 4 x 100m/400m 
                        row.append(dict_result2) # DICT_RESULT
                        row.append(file)
                        row.append('International')

                        
                      #  pass




                elif (event and dict_result[3] and dict_result[5]) is not None:

                   #

                    row.append(dict_result[0]) # rank

                    if '4 x 100m' not in event and '4 x 400m' not in event and 'Mixed Relay' not in event:

                        row.append(names[-1])  # choose last name in list as the correct one
                    else:
                        row.append(' ')



                   # row.append(dict_result[1]) # name  (dict_result[1])
                    row.append(dict_result[3]) # nationality
                    row.append(dict_result[5]) # result incl. DNS
                    row.append(dict_result[9]) # Q/q
                    row.append(competition)
                    row.append(year)
                    row.append(date)
                    row.append(event)
                    row.append(venue)
                    row.append(gender)
                    row.append(stage)
                    row.append(heat)
                    row.append(wind)
                    row.append(dict_result[4]) # DOB                    
                    row.append(dict_result[8]+' '+dict_result[2]) # Remarks. PB/SB
                    row.append(dict_result[10]) # Reaction time for 4 x 100m/400m 
                    row.append(dict_result2) # DICT_RESULT
                    row.append(file)
                    row.append('International')



                #print(row)

            except:

                pass

            if len(row)>5:

                print('ROW', row)


                temp_df.loc[len(temp_df)] = row


            print('LENGTH', len(lists))


            dict_result2 = []
            row=[]





        wind=''
        heat=''
        date=''

        lists=[]
        dict_result = []
        row=[]

                                  
    return temp_df   
                
            
            
            




In [200]:
# LATEST VERSION

def scraper_new(soup, file):

    year_pattern='20\d\d'
    dash_pattern='\-'

    matches = []
    lists=[]
    counts=[]
    values=[]
    names=[]
    wind=''
    stage=''
    event=''
    sub_event=''
    
    gender=''
    heat=''
    division = ''
    dict_result2=[]  # length 2 dict
    row=[]

    temp_df = pd.DataFrame(columns=['RANK', 'NAME', 'NATIONALITY', 'RESULT', 'QUALIFICATION', 'COMPETITION', 'YEAR', 'DATE', 'EVENT', 'VENUE', 'GENDER', 'STAGE', 'HEAT', 'WIND', 'DOB', 'DIVISION', 'REMARKS', 'RX_TIME', 'DICT_RESULT', 'SOURCE', 'REGION', 'HOST_CITY', 'SUB_EVENT'])

    for div in soup.find_all('div', attrs={ "class" : "left"}):  # find appropriate div

        info=div.find('h1')  # then find 'h1' to locate event info

        text=info.text

        list=text.split(',')
        
        # HOST CITY
    
        host_city = list[1]
    
        print('HOST CITY', host_city[0])
   
        result=list[0].split(':')

        competition=result[1]
        
      #  competition='Southeast Asian Games'  # Fix the competition for SEAG

      #  print('EVENT', competition)

       # year = re.findall(year_pattern, list[2])

        #print('YEAR', year)

      #  dates = extract_dates(text)

       # year=dates[0].year

        year_pos = re.search(year_pattern, text)

        year = re.findall(year_pattern, text)[0]

      #  date = dates[0].strftime("%B")+' '+str(dates[0].day)

        date1 = re.finditer(dash_pattern, text)


     #   print('YEAR', year)

        for match in date1:

        #    print('MATCH', match.span())

            matches.append(match.span())

        venue =' '

     #   print('MATCHES', matches[0][1])  # choose first tuple, second element

        date = text[matches[0][1]:year_pos.start()]  # extract date

     #   print('DATE', date)


    #h1s = soup.find_all('h1')

    #for h1 in h1s:

    #    text=h1.text

    #    if 'RESULTS' in text:

    #        list=text.split(',')

    #        list1=list[0].split(':')

    #        print('event', list1)

    # VENUE
    
    venue_text = div.find('h2')
    
    venue = venue_text.text
    
    print('VENUE', venue[7:])



    for i in soup.find_all('table'): # Look for the table first then find <tr> for each athlete

        for element in i.find_all('tr'): # <tr> separates each athlete/section

       #     print('NEW SECTION')

           # for td in element.find_all('td'):

            counts=[]
            values=[]


            for count, value in enumerate(element.find_all('td')):

                for n in value.find_all('a'):  # extract the name and put into a list

                    if len(n.text)>5 and '(' not in n.text and '1y' not in n.text and '/' not in n.text:    # take out all the miscellaneous           

                        names.append(n.text) # add 'clean' name to list

                for m in value.find_all('b'):  # extract heats/stage/division e.g. U20/U18 etc.

               #     print('NEW HEATS/STAGE/DIVISION', m.text)
            
                    if 'Multievents' in m.text:
                    
                        sub_event = 'Multievents'
                

                    if 'U20' in m.text or 'U18' in m.text or 'U17' in m.text or 'U16' in m.text or 'U15' in m.text or 'U14' in m.text: # From AUS state championships

                        division = m.text
                    else:

                        heat = m.text # NEW ADDITION for Nagase Cup

                    dict_result = ''  # NEW ADDITION for Nagase Cup. Clear dict_result before next heat


                if len(lists)==2:  # captures gender

                   # heat=''
                    wind=''
                    dict_result=''
                    dict_result2=''
                  #  stage=''

               #     print('RESET heat, stage HERE')

                    if '4 x 100m' not in event and '4 x 400m' not in event: # prevent reset of heats and stage unless relay. Relay reset for stage and heat different than other events

                        heat = ''
                        stage = ''

                    if 'Men' in lists[1] and len(lists[1])==3: # exact match for 'Men'
                        gender = 'Male'

                    if 'Women' in lists[1]:
                        gender = 'Female'


             #   if 'Date' in value.text:
             #       date=value.text

                if 'Wind' in value.text:

                    string=value.text
                    wind=string[5:]  # NEW ADDITION - Nagase Cup - slice out the 'WIND:'
               #     print('WINDDDDD', wind)

         #       print('TDTD', count, value.text) # extract Heat, Wind Rank, Nationality, DOB, Result, PB/SB

                counts.append(count)
                values.append(value.text)


            lists=dict(zip(counts,values)) # convert every TD line under each TR section into a dictionary


            if len(lists)==3 and 'Wind' in lists[2]:  # if index 2 does not contain 'wind' then it's a new stage

                heat = lists[1]
                wind = lists[2]  # wind can also appear in list of length 5

                if 'Wind' in lists[2]:

                    wind = lists[2][5:]  # slice out the 'Wind:'


                dict_result=[]

            if len(lists)==3 and 'Wind' not in lists[2]:


                if 'Semifinals' in lists[1]:

                    stage = 'Semifinals'

                if 'Heats' in lists[1]:

                    stage = 'Heats'

                if 'Heat 1' in lists[1]:

                    heat = 'Heat 1'

                if 'Heat 2' in lists[1]:

                    heat = 'Heat 2'

                if 'Heat 3' in lists[1]:

                    heat = 'Heat 3'

                if 'Heat 4' in lists[1]:

                    heat = 'Heat 4'


              #  if len(lists[2])>1:

              #      date = lists[2]  # set date only if lists[2] is not null

                dict_result = []

            if len(lists)==5:  # dict of length=5 contains date, event info. Reset Heat, Stage and Wind when new event ecountered

            #    date = lists[2]
                event = lists[1]
                wind = ' '
                stage = ' '
                heat = ' '

             #   print('Reset STAGE, HEAT here')

                dict_result = []
                dict_result2 = []

                if 'Wind' in lists[4]:

                    wind = lists[4][5:]  # slice out the 'Wind:'

               #     print('WINDWIND', wind)


            if len(lists)==11: # when a new athlete is encountered

                dict_result=lists



            #    dict_result2 = [] # reset attempts


            if len(lists)==2 and 'Men' not in lists[1]: # dict length 2 contains event gender info or attempts info
                dict_result2 = lists


            if len(lists)==2 and 'Women' not in lists[1]: # dict length 2 contains event gender info or attempts info
                dict_result2 = lists



            print('EVENT', event, 'GENDER', gender, 'STAGE', stage, 'HEAT', heat, 'WIND', wind, 'DATE', date , 'DIVISION', division, 'DICT_RESULT', dict_result, 'ATTEMPTS', dict_result2)

            try:

                if 'High Jump' in event or 'Pole Vault' in event or 'Long Jump' in event or 'Triple Jump' in event or 'Shot Put' in event or 'Discus Throw' in event or 'Hammer Throw' in event or 'Javelin Throw' in event or 'Decathlon' in event or '4 x 100m' in event or '4 x 400m' in event or 'Heptathlon' in event:



                    if len(dict_result2)>0:

                    #    print('PREVIOUS ROW', temp_df.tail(1))


                       # pass
                      #  temp_df.drop(temp_df.tail(1).index,inplace = True)  # drop previous row without the list of attempts


                 #       temp_df.loc[row_index,'DICT_RESULTS'] = dict_result2



                        row.append(dict_result[0]) # rank

                        if '4 x 100m' not in event and '4 x 400m' not in event and 'Mixed Relay' not in event:

                            row.append(names[-1])
                        else:
                            row.append(' ')  # name is blank for relays

                        remarks=dict_result[8]+ ' ' + dict_result[2]

                        if 'PB' in remarks:

                            remarks='PB'

                        elif 'SB' in remarks:

                            remarks='SB'


                        row.append(dict_result[3]) # nationality
                        row.append(dict_result[5]) # result incl. DNS
                        row.append(dict_result[9]) # Q/q
                        row.append(competition)
                        row.append(year)
                        row.append(date)
                        row.append(event)
                        row.append(venue[7:])
                        row.append(gender)
                        row.append(stage)
                        row.append(heat)
                        row.append(wind)
                        row.append(dict_result[4]) # DOB
                        row.append(division)
                        row.append(remarks) # Remarks. PB/SB
                        row.append(dict_result[10]) # Reaction time for 4 x 100m/400m 
                        row.append(dict_result2) # DICT_RESULT
                        row.append(file)
                        row.append('International')
                        row.append(host_city)
                        row.append(sub_event)



                    elif len(dict_result2)==0:  # append row but do not delete previous row

                        row.append(dict_result[0]) # rank

                        if '4 x 100m' not in event and '4 x 400m' not in event and 'Mixed Relay' not in event:

                            row.append(names[-1])
                        else:
                             row.append(' ')  # name is blank for relays

                        remarks=dict_result[8]+ ' ' + dict_result[2]

                        if 'PB' in remarks:

                            remarks='PB'

                        elif 'SB' in remarks:

                            remarks='SB'


                        row.append(dict_result[3]) # nationality
                        row.append(dict_result[5]) # result incl. DNS
                        row.append(dict_result[9]) # Q/q
                        row.append(competition)
                        row.append(year)
                        row.append(date)
                        row.append(event)
                        row.append(venue[7:])
                        row.append(gender)
                        row.append(stage)
                        row.append(heat)
                        row.append(wind)
                        row.append(dict_result[4]) # DOB
                        row.append(division)
                        row.append(remarks) # Remarks. PB/SB
                        row.append(dict_result[10]) # Reaction time for 4 x 100m/400m 
                        row.append(dict_result2) # DICT_RESULT
                        row.append(file)
                        row.append('International')
                        row.append(host_city)
                        row.append(sub_event)

                      #  pass




                elif (event and dict_result[3] and dict_result[5]) is not None:

                   #

                    row.append(dict_result[0]) # rank

                    if '4 x 100m' not in event and '4 x 400m' not in event and 'Mixed Relay' not in event:

                        row.append(names[-1])  # choose last name in list as the correct one
                    else:
                        row.append(' ')

                    remarks=dict_result[8]+ ' ' + dict_result[2]

                    if 'PB' in remarks:

                        remarks='PB'

                    elif 'SB' in remarks:

                        remarks='SB'


                    row.append(dict_result[3]) # nationality
                    row.append(dict_result[5]) # result incl. DNS
                    row.append(dict_result[9]) # Q/q
                    row.append(competition)
                    row.append(year)
                    row.append(date)
                    row.append(event)
                    row.append(venue[7:])
                    row.append(gender)
                    row.append(stage)
                    row.append(heat)
                    row.append(wind)
                    row.append(dict_result[4]) # DOB 
                    row.append(division)
                    row.append(remarks) # Remarks. PB/SB
                    row.append(dict_result[10]) # Reaction time for 4 x 100m/400m 
                    row.append(dict_result2) # DICT_RESULT
                    row.append(file)
                    row.append('International')
                    row.append(host_city)
                    row.append(sub_event)



            except:

                pass

            if len(row)>5:

           #     print('ADDED ROW to DF', row)


                temp_df.loc[len(temp_df)] = row


           #     print('LENGTH DF', len(temp_df))


         #   print('LENGTH', len(lists))


            dict_result2 = []
            row=[]

        wind=''
        heat=''
   #     date=''


        lists=[]
        dict_result = []
        row=[]

    return temp_df

In [216]:
# LATEST VERSON
# Test open all files in directory

#os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Tilastopaja/Asian Athletics Championship/')
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Tilastopaja/Singapore Athletes Events/2025/Feb/Late additions/')


def sort_directory(directory):
    items = os.listdir(directory)
    sorted_items = sorted(items)
    return sorted_items



directory = r"/Users/veesheenyuen/Desktop/DataScience/SAA/Tilastopaja/Singapore Athletes Events/2025/Feb/Late additions"
#directory = r"/Users/veesheenyuen/Desktop/DataScience/SAA/Tilastopaja/Asian Athletics Championship/"
 
    
    
# Iterate over files in directory

sorted_items = sort_directory(directory)

files = sorted_items

print('number', len(sorted_items))

files_clean = [ file for file in files if file.endswith(".html") ]

#sorted_items.pop(0)  # remove first item (DS_store)

master_df=pd.DataFrame()  # initialize empty master df



for file in files_clean:
    
    print(file)
    
    HTMLFileToBeOpened = open(file, "r") 

    contents = HTMLFileToBeOpened.read() 
    soup = BeautifulSoup(contents, 'lxml') 
    
    temp_df=scraper_new(soup, file)    
    
    
    master_df=pd.concat([master_df, temp_df], axis=0)
    


number 3
Tilastopaja - Dubai World Para Athletics Grand Prix - Feb 25.html
HOST CITY  
VENUE events: 
EVENT  GENDER  STAGE  HEAT  WIND  DATE  10 - 13 February  DIVISION  DICT_RESULT [] ATTEMPTS []
EVENT  GENDER  STAGE  HEAT F11 WIND  DATE  10 - 13 February  DIVISION  DICT_RESULT  ATTEMPTS {0: '\xa0', 1: 'F11'}
EVENT  GENDER  STAGE  HEAT  WIND  DATE  10 - 13 February  DIVISION  DICT_RESULT  ATTEMPTS {0: '\xa0', 1: 'Men'}
EVENT Shot Put     GENDER Male STAGE   HEAT   WIND   DATE  10 - 13 February  DIVISION  DICT_RESULT [] ATTEMPTS []
EVENT Shot Put     GENDER Male STAGE   HEAT   WIND   DATE  10 - 13 February  DIVISION  DICT_RESULT {0: '1', 1: "Amirhossein\xa0Alipour DarbeidAmirhossein\xa0Alipour Darbeid\xa0('02)", 2: '', 3: 'IRI', 4: '2 Mar 02', 5: '14.59 ', 6: '', 7: '\xa0', 8: 'PB »', 9: '', 10: 'F11/F12/F20     '} ATTEMPTS []
EVENT Shot Put     GENDER Male STAGE   HEAT   WIND   DATE  10 - 13 February  DIVISION  DICT_RESULT {0: '1', 1: "Amirhossein\xa0Alipour DarbeidAmirhossein\xa0Alip

EVENT Discus Throw     GENDER Male STAGE   HEAT   WIND   DATE  10 - 13 February  DIVISION  DICT_RESULT {0: '6', 1: "Rashid\xa0AlshamsiRashid\xa0Alshamsi\xa0('97)", 2: '', 3: 'UAE', 4: '97', 5: '11.14 ', 6: '', 7: '\xa0', 8: 'PB »', 9: '', 10: 'F51/F52/F54     '} ATTEMPTS []
EVENT Javelin Throw     GENDER Male STAGE   HEAT   WIND   DATE  10 - 13 February  DIVISION  DICT_RESULT [] ATTEMPTS []
EVENT Javelin Throw     GENDER Male STAGE   HEAT   WIND   DATE  10 - 13 February  DIVISION  DICT_RESULT {0: '4', 1: "Abdulla\xa0MesbahiAbdulla\xa0Mesbahi\xa0('85)", 2: '', 3: 'UAE', 4: '85', 5: '18.10 ', 6: '', 7: '\xa0', 8: 'PB »', 9: '', 10: 'F33/F53/F54     '} ATTEMPTS []
EVENT Javelin Throw     GENDER Male STAGE   HEAT   WIND   DATE  10 - 13 February  DIVISION  DICT_RESULT {0: '6', 1: "Mohammad Wazid Ali\xa0Mohammad Wazid Ali\xa0\xa0('89)", 2: '', 3: 'IND', 4: '89', 5: '14.34 ', 6: '', 7: '\xa0', 8: 'PB »', 9: '', 10: 'F33/F53/F54     '} ATTEMPTS []
EVENT Javelin Throw     GENDER Male STAGE   HE

In [217]:
master_df

,RANK,NAME,NATIONALITY,RESULT,QUALIFICATION,COMPETITION,YEAR,DATE,EVENT,VENUE,...,WIND,DOB,DIVISION,REMARKS,RX_TIME,DICT_RESULT,SOURCE,REGION,HOST_CITY,SUB_EVENT
0,1,Amirhossein Alipour Darbeid,IRI,14.59,,Dubai World Para Athletics Grand Prix,2025,10 - 13 February,Shot Put,events:,...,,2 Mar 02,,PB,F11/F12/F20,[],Tilastopaja - Dubai World Para Athletics Grand...,International,16th Fazza International Open Championship,
1,1,Amirhossein Alipour Darbeid,IRI,14.59,,Dubai World Para Athletics Grand Prix,2025,10 - 13 February,Shot Put,events:,...,,2 Mar 02,,PB,F11/F12/F20,[],Tilastopaja - Dubai World Para Athletics Grand...,International,16th Fazza International Open Championship,
2,2,Hassan Bajoulvand,IRI,12.05,,Dubai World Para Athletics Grand Prix,2025,10 - 13 February,Shot Put,events:,...,,6 Mar 85,,PB,F11/F12/F20,[],Tilastopaja - Dubai World Para Athletics Grand...,International,16th Fazza International Open Championship,
3,2,Hassan Bajoulvand,IRI,12.05,,Dubai World Para Athletics Grand Prix,2025,10 - 13 February,Shot Put,events:,...,,6 Mar 85,,PB,F11/F12/F20,[],Tilastopaja - Dubai World Para Athletics Grand...,International,16th Fazza International Open Championship,
4,3,Rajab Chetty,KEN,10.80,,Dubai World Para Athletics Grand Prix,2025,10 - 13 February,Shot Put,events:,...,,7 May 90,,PB,F11/F12/F20,[],Tilastopaja - Dubai World Para Athletics Grand...,International,16th Fazza International Open Championship,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
528,3,Lisa Schultis,SUI,1:58.94,,Dubai World Para Athletics Grand Prix,2025,10 - 13 February,800m,events:,...,,97,,PB,T54,[],Tilastopaja - Dubai World Para Athletics Grand...,International,16th Fazza International Open Championship,
529,1,Patricia Eachus,SUI,11:35.86,,Dubai World Para Athletics Grand Prix,2025,10 - 13 February,5000m,events:,...,,26 Sep 89,,PB,T54,[],Tilastopaja - Dubai World Para Athletics Grand...,International,16th Fazza International Open Championship,
530,1,Ali Lacin,GER,5.57,,Dubai World Para Athletics Grand Prix,2025,10 - 13 February,Long Jump,events:,...,,17 Apr 88,,SB,T61/T63/T64,[],Tilastopaja - Dubai World Para Athletics Grand...,International,16th Fazza International Open Championship,
531,2,Rashed Almuharrami,UAE,33.77w,,Dubai World Para Athletics Grand Prix,2025,10 - 13 February,100m,events:,...,2.1,02,,,T71/T72,[],Tilastopaja - Dubai World Para Athletics Grand...,International,16th Fazza International Open Championship,


In [218]:
# Remove special characters
    
for col in master_df.columns:
    master_df[col] = master_df[col].astype(str)
    master_df[col] = master_df[col].str.replace('\xa0', ' ', regex=True)
    master_df[col] = master_df[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    master_df[col] = master_df[col].str.replace('\r', ' ', regex=True)
    master_df[col] = master_df[col].str.replace('\n', ' ', regex=True)
    master_df[col] = master_df[col].str.strip()


    


In [219]:
# removes duplicate rows from those with list of attempts

master_df = master_df.drop_duplicates(['RANK', 'NAME', 'NATIONALITY', 'RESULT', 'QUALIFICATION', 'COMPETITION', 'YEAR', 'DATE', 'EVENT', 'VENUE', 'GENDER', 'STAGE', 'HEAT', 'WIND', 'DOB', 'REMARKS', 'RX_TIME', 'HOST_CITY'], keep='last')

In [220]:
# removes duplicate rows from those with list of attempts for relays

master_df = master_df.drop_duplicates(['RANK', 'NAME', 'NATIONALITY', 'RESULT', 'QUALIFICATION', 'COMPETITION', 'YEAR', 'DATE', 'EVENT', 'VENUE', 'GENDER', 'STAGE', 'HEAT', 'WIND', 'DOB', 'REMARKS', 'RX_TIME', 'HOST_CITY'], keep='last')

In [221]:
# Map general event category 

master_df.loc[master_df['EVENT'].str.contains(r'^100m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Sprint'
master_df.loc[master_df['EVENT'].str.contains(r'^400m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Sprint'
master_df.loc[master_df['EVENT'].str.contains(r'^600m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Mid'
master_df.loc[master_df['EVENT'].str.contains(r'^60m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Sprint'
master_df.loc[master_df['EVENT'].str.contains(r'^200m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Sprint'
master_df.loc[master_df['EVENT'].str.contains(r'50m', regex=True, na=True), 'CATEGORY_EVENT'] = 'Sprint'
master_df.loc[master_df['EVENT'].str.contains(r'80m', regex=True, na=True), 'CATEGORY_EVENT'] = 'Sprint'
master_df.loc[master_df['EVENT'].str.contains(r'300m', regex=True, na=True), 'CATEGORY_EVENT'] = 'Sprint'
master_df.loc[master_df['EVENT'].str.contains(r'^3000m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Long'
master_df.loc[master_df['EVENT'].str.contains(r'Mile', regex=True, na=True), 'CATEGORY_EVENT'] = 'Mid'
master_df.loc[master_df['EVENT'].str.contains(r'^5000m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Long'
master_df.loc[master_df['EVENT'].str.contains(r'^2000m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Mid'
master_df.loc[master_df['EVENT'].str.contains(r'^1000m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Mid'
master_df.loc[master_df['EVENT'].str.contains(r'^800m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Mid'
master_df.loc[master_df['EVENT'].str.contains(r'^1500m$', regex=True, na=True), 'CATEGORY_EVENT'] = 'Mid'
master_df.loc[master_df['EVENT'].str.contains(r'10,000m', regex=True, na=True), 'CATEGORY_EVENT'] = 'Long'
master_df.loc[master_df['EVENT'].str.contains(r'5km', regex=True, na=True), 'CATEGORY_EVENT'] = 'Long'
master_df.loc[master_df['EVENT'].str.contains(r'10km', regex=True, na=True), 'CATEGORY_EVENT'] = 'Long'
master_df.loc[master_df['EVENT'].str.contains(r'Cross Country', na=True), 'CATEGORY_EVENT'] = 'Cross Country'


# Override values

master_df.loc[master_df['EVENT'].str.contains(r'Discus', na=True), 'CATEGORY_EVENT'] = 'Throw'
master_df.loc[master_df['EVENT'].str.contains(r'Javelin', na=True), 'CATEGORY_EVENT'] = 'Throw'
master_df.loc[master_df['EVENT'].str.contains(r'Shot', na=True), 'CATEGORY_EVENT'] = 'Throw'
master_df.loc[master_df['EVENT'].str.contains(r'Vault', na=True), 'CATEGORY_EVENT'] = 'Jump'
master_df.loc[master_df['EVENT'].str.contains(r'Throw', na=True), 'CATEGORY_EVENT'] = 'Throw'
master_df.loc[master_df['EVENT'].str.contains(r'Jump', na=True), 'CATEGORY_EVENT'] = 'Jump'
master_df.loc[master_df['EVENT'].str.contains(r'Relay', na=True), 'CATEGORY_EVENT'] = 'Relay'
master_df.loc[master_df['EVENT'].str.contains(r'Steeple', na=True), 'CATEGORY_EVENT'] = 'Steeple'
master_df.loc[master_df['EVENT'].str.contains(r'Walk', na=True), 'CATEGORY_EVENT'] = 'Walk'
master_df.loc[master_df['EVENT'].str.contains(r'Hurdles', na=True), 'CATEGORY_EVENT'] = 'Hurdles'
master_df.loc[master_df['EVENT'].str.contains(r'Pentathlon', na=True), 'CATEGORY_EVENT'] = 'Pentathlon'
master_df.loc[master_df['EVENT'].str.contains(r'Heptathlon', na=True), 'CATEGORY_EVENT'] = 'Heptathlon'
master_df.loc[master_df['EVENT'].str.contains(r'Triathlon', na=True), 'CATEGORY_EVENT'] = 'Triathlon'
master_df.loc[master_df['EVENT'].str.contains(r'Decathlon', na=True), 'CATEGORY_EVENT'] = 'Decathlon'
master_df.loc[master_df['EVENT'].str.contains(r'Marathon', na=True), 'CATEGORY_EVENT'] = 'Marathon'
master_df.loc[master_df['EVENT'].str.contains(r'4 x 100m', na=True), 'CATEGORY_EVENT'] = 'Relay'
master_df.loc[master_df['EVENT'].str.contains(r'4 x 400m', na=True), 'CATEGORY_EVENT'] = 'Relay'

# Copy relay event dictionary names into NAME column

master_df['NAME'] = np.where(((master_df['NAME'] == '') & ((master_df['EVENT']=='4 x 100m')|(master_df['EVENT']=='4 x 400m'))), master_df['DICT_RESULT'], master_df['NAME'])

In [222]:
# Convert to NEW SCHEMA



master_df['LAST_NAME'] = ''
master_df['FIRST_NAME'] = ''
master_df['OTHER_NAME'] = ''
master_df['TEAM'] = ''
master_df['SEED'] = ''
master_df['LANE'] = ''
master_df['DIVISION'] = ''
master_df['AGE'] = ''
master_df['UNIQUE_ID'] = ''
master_df['ATHLETE_ID'] = ''
master_df['TIMESTAMP'] = ''
master_df['TAG_ID'] = ''
master_df['POINTS'] = ''
master_df['GROUP'] = ''
master_df['SESSION']=''
master_df['EVENT_CLASS']=''
master_df['DISTANCE']=''


master_df = master_df.reindex(columns= ['FIRST_NAME', 'LAST_NAME', 'OTHER_NAME', 'NAME', 'RANK', 'TAG_ID', 'TEAM', 'SEED', 'RESULT', 'QUALIFICATION',
                                        'HEAT', 'LANE', 'WIND', 'EVENT', 'DIVISION', 'STAGE', 'POINTS', 'AGE', 'GENDER', 'UNIQUE_ID', 'NATIONALITY',
                                        'DICT_RESULTS', 'YEAR', 'DATE', 'COMPETITION', 'REGION', 'DOB', 'GROUP', 'CATEGORY_EVENT', 'ATHLETE_ID',
                                        'SOURCE', 'REMARKS', 'TIMESTAMP', 'VENUE', 'SUB_EVENT', 'SESSION', 'EVENT_CLASS', 'DISTANCE', 'HOST_CITY', 'RX_TIME'])


In [223]:
#master_df[master_df['NAME']=='Amir Rusyaidi Osman']

In [224]:
# Correct nationality for Amir

#mask = (master_df['NAME'].str.contains(r'Amir Rusyaidi Osman', na=True, regex=True) & (master_df['NATIONALITY']=='AUS'))# there are some additional characters after Put
#master_df.loc[mask, 'NATIONALITY'] = 'SGP'


In [225]:
#SEAG_manual_additions=master_df[master_df['EVENT']=='Marathon']

In [226]:
#master_df.to_csv('May_2025_first_batch.csv', encoding='utf-8')

In [231]:
SGP_athletes=master_df[master_df['NATIONALITY']=='SGP']

SGP_athletes.reset_index(drop=True, inplace=True)


In [232]:
SGP_athletes

,FIRST_NAME,LAST_NAME,OTHER_NAME,NAME,RANK,TAG_ID,TEAM,SEED,RESULT,QUALIFICATION,...,SOURCE,REMARKS,TIMESTAMP,VENUE,SUB_EVENT,SESSION,EVENT_CLASS,DISTANCE,HOST_CITY,RX_TIME
0,,,,Maveric Dong En Lee,3,,,,59.18,,...,Tilastopaja - Dubai World Para Athletics Grand...,PB,,events:,,,,,16th Fazza International Open Championship,T20
1,,,,Siti Nurhayati Ali Aksar,3,,,,1:10.33,,...,Tilastopaja - Dubai World Para Athletics Grand...,PB,,events:,,,,,16th Fazza International Open Championship,T20
2,,,,Muhammad Diroy Noordin,4,,,,8.36,,...,Tilastopaja - Dubai World Para Athletics Grand...,PB,,events:,,,,,16th Fazza International Open Championship,F40/F42/F46/F63
3,,,,Maisarah Mohamed Hassan,3,,,,16.46w,,...,Tilastopaja - Dubai World Para Athletics Grand...,,,events:,,,,,16th Fazza International Open Championship,T36
4,,,,Maisarah Mohamed Hassan,5,,,,37.98,,...,Tilastopaja - Dubai World Para Athletics Grand...,PB,,events:,,,,,16th Fazza International Open Championship,T35/T36/T37


In [214]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/Tilastopaja/Singapore Athletes Events/2025/Feb/Late additions/')

SGP_athletes.to_csv("SGP_athletes_Feb_2025_late_addition.csv", encoding="utf-8")

In [215]:
SGP_athletes

,FIRST_NAME,LAST_NAME,OTHER_NAME,NAME,RANK,TAG_ID,TEAM,SEED,RESULT,QUALIFICATION,...,SOURCE,REMARKS,TIMESTAMP,VENUE,SUB_EVENT,SESSION,EVENT_CLASS,DISTANCE,HOST_CITY,RX_TIME
94,,,,Maveric Dong En Lee,3,,,,59.18,,...,Tilastopaja - Dubai World Para Athletics Grand...,PB,,events:,,,,,16th Fazza International Open Championship,T20
115,,,,Siti Nurhayati Ali Aksar,3,,,,1:10.33,,...,Tilastopaja - Dubai World Para Athletics Grand...,PB,,events:,,,,,16th Fazza International Open Championship,T20
324,,,,Muhammad Diroy Noordin,4,,,,8.36,,...,Tilastopaja - Dubai World Para Athletics Grand...,PB,,events:,,,,,16th Fazza International Open Championship,F40/F42/F46/F63
426,,,,Maisarah Mohamed Hassan,3,,,,16.46w,,...,Tilastopaja - Dubai World Para Athletics Grand...,,,events:,,,,,16th Fazza International Open Championship,T36
431,,,,Maisarah Mohamed Hassan,5,,,,37.98,,...,Tilastopaja - Dubai World Para Athletics Grand...,PB,,events:,,,,,16th Fazza International Open Championship,T35/T36/T37


# Read and process manually cleaned data

In [753]:
# read manually cleaned data

SGP_athletes = pd.read_csv("SGP_athletes_cleaned.csv")

In [754]:
SGP_athletes

,Unnamed: 0,RANK,NAME,NATIONALITY,RESULT,QUALIFICATION,COMPETITION,YEAR,DATE,EVENT,...,GENDER,STAGE,HEAT,WIND,DOB,REMARKS,RX_TIME,DICT_RESULT,SOURCE,REGION
0,102,4.0,Jun Jie Calvin Quek,SGP,53.19,NaN,UAE Athletics Grand Prix,2024,,400m Hurdles,...,Male,,,,26 Feb 96,« »,,[],Tilastopaja - UAE Athletics Grand Prix - May ...,International
1,31,2.0,Chui Ling Goh,SGP,17:33.73,NaN,11. Münchner Abendsportfest /MUC HochschulMS,2024,,5000m,...,Female,,,,27 Nov 92,« »,Mixed competition,[],Tilastopaja - 11. Münchner Abendsportfest :MU...,International
2,121,4.0,Chen Xiang Ang,SGP,8.01,NaN,11th Asian Indoor Athletics Championships,2024,17 February,60m Hurdles,...,Male,Heats,Heat 1,,3 Jul 94,SB «,,[],Tilastopaja - 11th Asian Indoor Athletics Cham...,International
3,112,8.0,Jun Jie Calvin Quek,SGP,52.05,NaN,11th Kinami Michitaka Memorial Athletics Meet,2024,,400m Hurdles,...,Male,,,,26 Feb 96,SB «,,[],Tilastopaja - 11th Kinami Michitaka Memorial A...,International
4,482,4.0,James Ethan Kai Meng Ang,SGP,52.55,NaN,11th Para Athletics World Championships,2024,22 May,400m,...,Male,Heats,Heat 2,,26 May 01,« »,0.187,[],Tilastopaja - 11th Para Athletics World Champi...,International
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,127,3.0,Jun Jie Calvin Quek,SGP,56.95,NaN,XXXIII Qosanov Memorial,2024,22 June,400m Hurdles,...,Male,Heats,Heat 2,,26 Feb 96,« »,,[],Tilastopaja - XXXIII Qosanov Memorial - Jun 24...,International
359,148,4.0,Kangli Emery Conrad,SGP,7.22,NaN,XXXIII Qosanov Memorial,2024,22 June,Long Jump,...,Male,NaN,NaN,NaN,26 Apr 06,« »,,[],Tilastopaja - XXXIII Qosanov Memorial - Jun 24...,International
360,320,5.0,Tia Louise Rozario,SGP,5.71,NaN,XXXIII Qosanov Memorial,2024,22 June,Long Jump,...,Female,NaN,NaN,NaN,14 Oct 00,« »,,[],Tilastopaja - XXXIII Qosanov Memorial - Jun 24...,International
361,330,5.0,Tia Louise Rozario,SGP,12.45,NaN,XXXIII Qosanov Memorial,2024,23 June,Triple Jump,...,Female,NaN,NaN,NaN,14 Oct 00,« »,,[],Tilastopaja - XXXIII Qosanov Memorial - Jun 24...,International


In [755]:
# Map general event category 

SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'100m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'400m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'600m', na=True), 'CATEGORY_EVENT'] = 'Mid'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'60m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'200m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'50m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'80m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'300m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'3000m', na=True), 'CATEGORY_EVENT'] = 'Long'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Mile', na=True), 'CATEGORY_EVENT'] = 'Mid'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'5000m', na=True), 'CATEGORY_EVENT'] = 'Long'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'2000m', na=True), 'CATEGORY_EVENT'] = 'Mid'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'800m', na=True), 'CATEGORY_EVENT'] = 'Mid'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'1500m', na=True), 'CATEGORY_EVENT'] = 'Mid'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'10,000m', na=True), 'CATEGORY_EVENT'] = 'Long'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'5km', na=True), 'CATEGORY_EVENT'] = 'Long'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'10km', na=True), 'CATEGORY_EVENT'] = 'Long'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Cross Country', na=True), 'CATEGORY_EVENT'] = 'Cross Country'


# Override values

SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Discus', na=True), 'CATEGORY_EVENT'] = 'Throw'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Javelin', na=True), 'CATEGORY_EVENT'] = 'Throw'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Shot', na=True), 'CATEGORY_EVENT'] = 'Throw'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Vault', na=True), 'CATEGORY_EVENT'] = 'Jump'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Throw', na=True), 'CATEGORY_EVENT'] = 'Throw'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Jump', na=True), 'CATEGORY_EVENT'] = 'Jump'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Relay', na=True), 'CATEGORY_EVENT'] = 'Relay'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Steeple', na=True), 'CATEGORY_EVENT'] = 'Steeple'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Walk', na=True), 'CATEGORY_EVENT'] = 'Walk'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Hurdles', na=True), 'CATEGORY_EVENT'] = 'Hurdles'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Pentathlon', na=True), 'CATEGORY_EVENT'] = 'Pentathlon'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Triathlon', na=True), 'CATEGORY_EVENT'] = 'Triathlon'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Decathlon', na=True), 'CATEGORY_EVENT'] = 'Decathlon'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'Marathon', na=True), 'CATEGORY_EVENT'] = 'Marathon'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'4 x 100m', na=True), 'CATEGORY_EVENT'] = 'Relay'
SGP_athletes.loc[SGP_athletes['EVENT'].str.contains(r'4 x 400m', na=True), 'CATEGORY_EVENT'] = 'Relay'




In [756]:
# Convert to OLD SCHEMA

#SGP_athletes['TEAM'] = 'Singapore'
#SGP_athletes['SEED'] = ''
#SGP_athletes['LANE'] = ''
#SGP_athletes['DIVISION'] = ''
#SGP_athletes['AGE'] = ''
#SGP_athletes['UNIQUE_ID'] = ''
#SGP_athletes['ATHLETE_ID'] = ''
#SGP_athletes['TIMESTAMP'] = ''
#SGP_athletes['TAG_ID'] = ''
#SGP_athletes['POINTS'] = ''
#SGP_athletes['GROUP'] = ''
#SGP_athletes['FREE_FIELD']=''
#SGP_athletes['FREE_FIELD2']=''
#SGP_athletes['FREE_FIELD3']=''


In [ ]:
# Convert to NEW SCHEMA

SGP_athletes['LAST_NAME'] = ''
SGP_athletes['FIRST_NAME'] = ''
SGP_athletes['OTHER_NAME'] = ''
SGP_athletes['TEAM'] = 'Singapore'
SGP_athletes['SEED'] = ''
SGP_athletes['LANE'] = ''
SGP_athletes['DIVISION'] = ''
SGP_athletes['AGE'] = ''
SGP_athletes['UNIQUE_ID'] = ''
SGP_athletes['ATHLETE_ID'] = ''
SGP_athletes['TIMESTAMP'] = ''
SGP_athletes['TAG_ID'] = ''
SGP_athletes['POINTS'] = ''
SGP_athletes['GROUP'] = ''
SGP_athletes['VENUE']=''
SGP_athletes['SUB_EVENT']=''
SGP_athletes['SESSION']=''
SGP_athletes['EVENT_CLASS']=''
SGP_athletes['SUB_EVENT']=''
SGP_athletes['DISTANCE']=''
SGP_athletes['RX_TIME']=''
SGP_athletes['CATEGORY_EVENT']=''


In [757]:
SGP_athletes

,Unnamed: 0,RANK,NAME,NATIONALITY,RESULT,QUALIFICATION,COMPETITION,YEAR,DATE,EVENT,...,AGE,UNIQUE_ID,ATHLETE_ID,TIMESTAMP,TAG_ID,POINTS,GROUP,FREE_FIELD,FREE_FIELD2,FREE_FIELD3
0,102,4.0,Jun Jie Calvin Quek,SGP,53.19,NaN,UAE Athletics Grand Prix,2024,,400m Hurdles,...,,,,,,,,,,
1,31,2.0,Chui Ling Goh,SGP,17:33.73,NaN,11. Münchner Abendsportfest /MUC HochschulMS,2024,,5000m,...,,,,,,,,,,
2,121,4.0,Chen Xiang Ang,SGP,8.01,NaN,11th Asian Indoor Athletics Championships,2024,17 February,60m Hurdles,...,,,,,,,,,,
3,112,8.0,Jun Jie Calvin Quek,SGP,52.05,NaN,11th Kinami Michitaka Memorial Athletics Meet,2024,,400m Hurdles,...,,,,,,,,,,
4,482,4.0,James Ethan Kai Meng Ang,SGP,52.55,NaN,11th Para Athletics World Championships,2024,22 May,400m,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,127,3.0,Jun Jie Calvin Quek,SGP,56.95,NaN,XXXIII Qosanov Memorial,2024,22 June,400m Hurdles,...,,,,,,,,,,
359,148,4.0,Kangli Emery Conrad,SGP,7.22,NaN,XXXIII Qosanov Memorial,2024,22 June,Long Jump,...,,,,,,,,,,
360,320,5.0,Tia Louise Rozario,SGP,5.71,NaN,XXXIII Qosanov Memorial,2024,22 June,Long Jump,...,,,,,,,,,,
361,330,5.0,Tia Louise Rozario,SGP,12.45,NaN,XXXIII Qosanov Memorial,2024,23 June,Triple Jump,...,,,,,,,,,,


In [758]:
df = SGP_athletes.drop(['VENUE', 'YEAR', 'RX_TIME'], axis=1)
df['DATE']='2024'

In [759]:
df

,Unnamed: 0,RANK,NAME,NATIONALITY,RESULT,QUALIFICATION,COMPETITION,DATE,EVENT,GENDER,...,AGE,UNIQUE_ID,ATHLETE_ID,TIMESTAMP,TAG_ID,POINTS,GROUP,FREE_FIELD,FREE_FIELD2,FREE_FIELD3
0,102,4.0,Jun Jie Calvin Quek,SGP,53.19,NaN,UAE Athletics Grand Prix,2024,400m Hurdles,Male,...,,,,,,,,,,
1,31,2.0,Chui Ling Goh,SGP,17:33.73,NaN,11. Münchner Abendsportfest /MUC HochschulMS,2024,5000m,Female,...,,,,,,,,,,
2,121,4.0,Chen Xiang Ang,SGP,8.01,NaN,11th Asian Indoor Athletics Championships,2024,60m Hurdles,Male,...,,,,,,,,,,
3,112,8.0,Jun Jie Calvin Quek,SGP,52.05,NaN,11th Kinami Michitaka Memorial Athletics Meet,2024,400m Hurdles,Male,...,,,,,,,,,,
4,482,4.0,James Ethan Kai Meng Ang,SGP,52.55,NaN,11th Para Athletics World Championships,2024,400m,Male,...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,127,3.0,Jun Jie Calvin Quek,SGP,56.95,NaN,XXXIII Qosanov Memorial,2024,400m Hurdles,Male,...,,,,,,,,,,
359,148,4.0,Kangli Emery Conrad,SGP,7.22,NaN,XXXIII Qosanov Memorial,2024,Long Jump,Male,...,,,,,,,,,,
360,320,5.0,Tia Louise Rozario,SGP,5.71,NaN,XXXIII Qosanov Memorial,2024,Long Jump,Female,...,,,,,,,,,,
361,330,5.0,Tia Louise Rozario,SGP,12.45,NaN,XXXIII Qosanov Memorial,2024,Triple Jump,Female,...,,,,,,,,,,


In [760]:
# Re-order columns

df = df.reindex(columns= ['RANK', 'TAG_ID', 'NAME', 'TEAM', 'SEED', 'RESULT', 'QUALIFICATION', 'HEAT', 'LANE', 'WIND', 'EVENT', 'DIVISION', 'STAGE', 
                                      'POINTS', 'AGE', 'GENDER', 'UNIQUE_ID', 'COUNTRY', 'DICT_RESULTS', 'DATE', 'COMPETITION', 'REGION', 'DOB','GROUP', 'CATEGORY_EVENT', 
                                      'ATHLETE_ID', 'SOURCE', 'REMARKS', 'TIMESTAMP', 'FREE_FIELD', 'FREE_FIELD2', 'FREE_FIELD3'])


In [761]:
df

,RANK,TAG_ID,NAME,TEAM,SEED,RESULT,QUALIFICATION,HEAT,LANE,WIND,...,DOB,GROUP,CATEGORY_EVENT,ATHLETE_ID,SOURCE,REMARKS,TIMESTAMP,FREE_FIELD,FREE_FIELD2,FREE_FIELD3
0,4.0,,Jun Jie Calvin Quek,Singapore,,53.19,NaN,,,,...,26 Feb 96,,Hurdles,,Tilastopaja - UAE Athletics Grand Prix - May ...,« »,,,,
1,2.0,,Chui Ling Goh,Singapore,,17:33.73,NaN,,,,...,27 Nov 92,,Long,,Tilastopaja - 11. Münchner Abendsportfest :MU...,« »,,,,
2,4.0,,Chen Xiang Ang,Singapore,,8.01,NaN,Heat 1,,,...,3 Jul 94,,Hurdles,,Tilastopaja - 11th Asian Indoor Athletics Cham...,SB «,,,,
3,8.0,,Jun Jie Calvin Quek,Singapore,,52.05,NaN,,,,...,26 Feb 96,,Hurdles,,Tilastopaja - 11th Kinami Michitaka Memorial A...,SB «,,,,
4,4.0,,James Ethan Kai Meng Ang,Singapore,,52.55,NaN,Heat 2,,,...,26 May 01,,Sprint,,Tilastopaja - 11th Para Athletics World Champi...,« »,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
358,3.0,,Jun Jie Calvin Quek,Singapore,,56.95,NaN,Heat 2,,,...,26 Feb 96,,Hurdles,,Tilastopaja - XXXIII Qosanov Memorial - Jun 24...,« »,,,,
359,4.0,,Kangli Emery Conrad,Singapore,,7.22,NaN,NaN,,NaN,...,26 Apr 06,,Jump,,Tilastopaja - XXXIII Qosanov Memorial - Jun 24...,« »,,,,
360,5.0,,Tia Louise Rozario,Singapore,,5.71,NaN,NaN,,NaN,...,14 Oct 00,,Jump,,Tilastopaja - XXXIII Qosanov Memorial - Jun 24...,« »,,,,
361,5.0,,Tia Louise Rozario,Singapore,,12.45,NaN,NaN,,NaN,...,14 Oct 00,,Jump,,Tilastopaja - XXXIII Qosanov Memorial - Jun 24...,« »,,,,


In [762]:
df.columns

Index(['RANK', 'TAG_ID', 'NAME', 'TEAM', 'SEED', 'RESULT', 'QUALIFICATION',
       'HEAT', 'LANE', 'WIND', 'EVENT', 'DIVISION', 'STAGE', 'POINTS', 'AGE',
       'GENDER', 'UNIQUE_ID', 'COUNTRY', 'DICT_RESULTS', 'DATE', 'COMPETITION',
       'REGION', 'DOB', 'GROUP', 'CATEGORY_EVENT', 'ATHLETE_ID', 'SOURCE',
       'REMARKS', 'TIMESTAMP', 'FREE_FIELD', 'FREE_FIELD2', 'FREE_FIELD3'],
      dtype='object')

In [763]:
# Remove special characters
    
for col in df.columns:
    df[col] = df[col].astype(str)
    df[col] = df[col].str.replace('\xa0', ' ', regex=True)
    df[col] = df[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    df[col] = df[col].str.replace('\r', ' ', regex=True)
    df[col] = df[col].str.replace('\n', ' ', regex=True)
    df[col] = df[col].str.strip()


    
    

In [764]:
df.reset_index(inplace=True)

In [765]:
df.to_csv('SG_athletes_international_2024.csv', encoding='utf-8')

# Process missing data extracted from SAA database

In [144]:
os.chdir('/Users/veesheenyuen/Desktop/DataScience/SAA/External events/')

athletes = pd.read_csv("ALL_external_data_from_SAA_database_Jan 2011_to_Dec_24.csv")

In [145]:
athletes

,Date,Event,Name,Age,Team,Result,Wind m/s,Competition,Year D.O.B.,"Info, if any"
0,19 Dec 2024,Men 800 Meter Run,"THANA RAJAN, THIRUBEN",24,SINGAPORE,1:53.51,-,International - Strive (Australia),2000,-
1,19 Dec 2024,Men 1500 Meter Run,"LIM, OLIVER",24,SINGAPORE,4:01.49,-,International - Vic Milers (Australia),1999,-
2,07 Dec 2024,Men 60 Meter Dash,"TAN, DARYL",23,SINGAPORE,6.84,1.4,"International - Strive Program B, Perth (Austr...",2001,-
3,07 Dec 2024,Men 200 Meter Dash,"TAN, DARYL",23,SINGAPORE,21.68,1.2,"International - Strive Program B, Perth (Austr...",2001,-
4,07 Dec 2024,Men 60 Meter Dash,"TOH JUN XI, TEDD",22,SINGAPORE,6.97,1.3,"International - Strive Program B, Perth (Austr...",2002,-
...,...,...,...,...,...,...,...,...,...,...
6535,16 Apr 2011,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,14.91,-,International - Australia Athletics Cships,1990,"- Preliminaries Olympic Park, Melbourne"
6536,27 Mar 2011,Women 10000 Meter Run,Renuka Satianathan,24,Singapore,37:15.87,-,International - Queensland Open & AWD Cships,1987,"- Finals State Athletics Facility, Nathan Quee..."
6537,27 Mar 2011,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,15.32,-,International - Athletic Victoria Throwers Mee...,1990,"- Finals Proclamations Park, Ringwood, Melbourne"
6538,11 Mar 2011,Men 5000 Meter Race Walk,Peter James Back,-,Singapore,26:53,HT,International - Thai Veterans Athletics Cships,-,"Open Final Bangkok, Thailand"


In [146]:
# Remove special characters
    
for col in athletes.columns:
    athletes[col] = athletes[col].astype(str)
    athletes[col] = athletes[col].str.replace('\xa0', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('[\x00-\x1f\x7f-\x9f]', '', regex=True)
    athletes[col] = athletes[col].str.replace('\r', ' ', regex=True)
    athletes[col] = athletes[col].str.replace('\n', ' ', regex=True)
    athletes[col] = athletes[col].str.strip()



In [147]:
athletes['REGION'] = 'International' 
athletes['SOURCE'] = 'SAA Database'



In [148]:
# convert Date column into datetime

athletes['Date']=pd.to_datetime(athletes['Date'], format='mixed')

athletes['Date']

0      2024-12-19
1      2024-12-19
2      2024-12-07
3      2024-12-07
4      2024-12-07
          ...    
6535   2011-04-16
6536   2011-03-27
6537   2011-03-27
6538   2011-03-11
6539   2011-02-20
Name: Date, Length: 6540, dtype: datetime64[ns]

In [149]:
# extract day and month only

athletes['date_extract']=athletes['Date'].dt.strftime('%d-%m')
athletes['YEAR']=athletes['Date'].dt.strftime('%Y')

In [150]:
athletes

,Date,Event,Name,Age,Team,Result,Wind m/s,Competition,Year D.O.B.,"Info, if any",REGION,SOURCE,date_extract,YEAR
0,2024-12-19,Men 800 Meter Run,"THANA RAJAN, THIRUBEN",24,SINGAPORE,1:53.51,-,International - Strive (Australia),2000,-,International,SAA Database,19-12,2024
1,2024-12-19,Men 1500 Meter Run,"LIM, OLIVER",24,SINGAPORE,4:01.49,-,International - Vic Milers (Australia),1999,-,International,SAA Database,19-12,2024
2,2024-12-07,Men 60 Meter Dash,"TAN, DARYL",23,SINGAPORE,6.84,1.4,"International - Strive Program B, Perth (Austr...",2001,-,International,SAA Database,07-12,2024
3,2024-12-07,Men 200 Meter Dash,"TAN, DARYL",23,SINGAPORE,21.68,1.2,"International - Strive Program B, Perth (Austr...",2001,-,International,SAA Database,07-12,2024
4,2024-12-07,Men 60 Meter Dash,"TOH JUN XI, TEDD",22,SINGAPORE,6.97,1.3,"International - Strive Program B, Perth (Austr...",2002,-,International,SAA Database,07-12,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6535,2011-04-16,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,14.91,-,International - Australia Athletics Cships,1990,"- Preliminaries Olympic Park, Melbourne",International,SAA Database,16-04,2011
6536,2011-03-27,Women 10000 Meter Run,Renuka Satianathan,24,Singapore,37:15.87,-,International - Queensland Open & AWD Cships,1987,"- Finals State Athletics Facility, Nathan Quee...",International,SAA Database,27-03,2011
6537,2011-03-27,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,15.32,-,International - Athletic Victoria Throwers Mee...,1990,"- Finals Proclamations Park, Ringwood, Melbourne",International,SAA Database,27-03,2011
6538,2011-03-11,Men 5000 Meter Race Walk,Peter James Back,-,Singapore,26:53,HT,International - Thai Veterans Athletics Cships,-,"Open Final Bangkok, Thailand",International,SAA Database,11-03,2011


In [151]:
mask = athletes['Event'].str.contains(r'Men', na=True)
athletes.loc[mask, 'GENDER'] = 'Male'

mask = athletes['Event'].str.contains(r'Women', na=True)
athletes.loc[mask, 'GENDER'] = 'Female'


In [152]:
athletes

,Date,Event,Name,Age,Team,Result,Wind m/s,Competition,Year D.O.B.,"Info, if any",REGION,SOURCE,date_extract,YEAR,GENDER
0,2024-12-19,Men 800 Meter Run,"THANA RAJAN, THIRUBEN",24,SINGAPORE,1:53.51,-,International - Strive (Australia),2000,-,International,SAA Database,19-12,2024,Male
1,2024-12-19,Men 1500 Meter Run,"LIM, OLIVER",24,SINGAPORE,4:01.49,-,International - Vic Milers (Australia),1999,-,International,SAA Database,19-12,2024,Male
2,2024-12-07,Men 60 Meter Dash,"TAN, DARYL",23,SINGAPORE,6.84,1.4,"International - Strive Program B, Perth (Austr...",2001,-,International,SAA Database,07-12,2024,Male
3,2024-12-07,Men 200 Meter Dash,"TAN, DARYL",23,SINGAPORE,21.68,1.2,"International - Strive Program B, Perth (Austr...",2001,-,International,SAA Database,07-12,2024,Male
4,2024-12-07,Men 60 Meter Dash,"TOH JUN XI, TEDD",22,SINGAPORE,6.97,1.3,"International - Strive Program B, Perth (Austr...",2002,-,International,SAA Database,07-12,2024,Male
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6535,2011-04-16,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,14.91,-,International - Australia Athletics Cships,1990,"- Preliminaries Olympic Park, Melbourne",International,SAA Database,16-04,2011,Male
6536,2011-03-27,Women 10000 Meter Run,Renuka Satianathan,24,Singapore,37:15.87,-,International - Queensland Open & AWD Cships,1987,"- Finals State Athletics Facility, Nathan Quee...",International,SAA Database,27-03,2011,Female
6537,2011-03-27,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,15.32,-,International - Athletic Victoria Throwers Mee...,1990,"- Finals Proclamations Park, Ringwood, Melbourne",International,SAA Database,27-03,2011,Male
6538,2011-03-11,Men 5000 Meter Race Walk,Peter James Back,-,Singapore,26:53,HT,International - Thai Veterans Athletics Cships,-,"Open Final Bangkok, Thailand",International,SAA Database,11-03,2011,Male


In [153]:
athletes['COMPETITION'] = athletes['Competition'].str[16:]  # Remove 'International -' from string

In [154]:
athletes

,Date,Event,Name,Age,Team,Result,Wind m/s,Competition,Year D.O.B.,"Info, if any",REGION,SOURCE,date_extract,YEAR,GENDER,COMPETITION
0,2024-12-19,Men 800 Meter Run,"THANA RAJAN, THIRUBEN",24,SINGAPORE,1:53.51,-,International - Strive (Australia),2000,-,International,SAA Database,19-12,2024,Male,Strive (Australia)
1,2024-12-19,Men 1500 Meter Run,"LIM, OLIVER",24,SINGAPORE,4:01.49,-,International - Vic Milers (Australia),1999,-,International,SAA Database,19-12,2024,Male,Vic Milers (Australia)
2,2024-12-07,Men 60 Meter Dash,"TAN, DARYL",23,SINGAPORE,6.84,1.4,"International - Strive Program B, Perth (Austr...",2001,-,International,SAA Database,07-12,2024,Male,"Strive Program B, Perth (Australia)"
3,2024-12-07,Men 200 Meter Dash,"TAN, DARYL",23,SINGAPORE,21.68,1.2,"International - Strive Program B, Perth (Austr...",2001,-,International,SAA Database,07-12,2024,Male,"Strive Program B, Perth (Australia)"
4,2024-12-07,Men 60 Meter Dash,"TOH JUN XI, TEDD",22,SINGAPORE,6.97,1.3,"International - Strive Program B, Perth (Austr...",2002,-,International,SAA Database,07-12,2024,Male,"Strive Program B, Perth (Australia)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6535,2011-04-16,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,14.91,-,International - Australia Athletics Cships,1990,"- Preliminaries Olympic Park, Melbourne",International,SAA Database,16-04,2011,Male,Australia Athletics Cships
6536,2011-03-27,Women 10000 Meter Run,Renuka Satianathan,24,Singapore,37:15.87,-,International - Queensland Open & AWD Cships,1987,"- Finals State Athletics Facility, Nathan Quee...",International,SAA Database,27-03,2011,Female,Queensland Open & AWD Cships
6537,2011-03-27,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,15.32,-,International - Athletic Victoria Throwers Mee...,1990,"- Finals Proclamations Park, Ringwood, Melbourne",International,SAA Database,27-03,2011,Male,Athletic Victoria Throwers Meet #6
6538,2011-03-11,Men 5000 Meter Race Walk,Peter James Back,-,Singapore,26:53,HT,International - Thai Veterans Athletics Cships,-,"Open Final Bangkok, Thailand",International,SAA Database,11-03,2011,Male,Thai Veterans Athletics Cships


In [155]:
mask = athletes['Info, if any'].str.contains(r'Final', na=True)
athletes.loc[mask, 'STAGE'] = 'Final'

mask = athletes['Info, if any'].str.contains(r'Prelim', na=True)
athletes.loc[mask, 'STAGE'] = 'Prelim'

mask = athletes['Info, if any'].str.contains(r'Semifinal', na=True)
athletes.loc[mask, 'STAGE'] = 'Semi'

mask = athletes['Info, if any'].str.contains(r'Semi', na=True)
athletes.loc[mask, 'STAGE'] = 'Semi'


mask = athletes['Info, if any'].str.contains(r'Heats', na=True)
athletes.loc[mask, 'STAGE'] = 'Heats'


In [156]:
mask = athletes['Info, if any'].str.contains(r'U20', na=True)
athletes.loc[mask, 'DIVISION'] = 'U20'

mask = athletes['Info, if any'].str.contains(r'U18', na=True)
athletes.loc[mask, 'DIVISION'] = 'U18'


In [157]:
mask = athletes['Info, if any'].str.contains(r'PB', na=True)
athletes.loc[mask, 'REMARKS'] = 'PB'

mask = athletes['Info, if any'].str.contains(r'SB', na=True)
athletes.loc[mask, 'REMARKS'] = 'SB'

mask = athletes['Info, if any'].str.contains(r'CR', na=True)
athletes.loc[mask, 'REMARKS'] = 'CR'

mask = athletes['Info, if any'].str.contains(r'NR', na=True)
athletes.loc[mask, 'REMARKS'] = 'NR'

mask = athletes['Info, if any'].str.contains(r'R1', na=True)
athletes.loc[mask, 'REMARKS'] = 'R1'


In [158]:
athletes.rename(columns={'Event': 'EVENT'}, inplace=True)


In [159]:
# Running

mask = athletes['EVENT'].str.contains(r'50 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '50m'
mask = athletes['EVENT'].str.contains(r'60 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '60m'
mask = athletes['EVENT'].str.contains(r'80 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '80m'
mask = athletes['EVENT'].str.contains(r'100 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'100 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'^100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m'
mask = athletes['EVENT'].str.contains(r'200 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'^200m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'200\sMeter', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m'
mask = athletes['EVENT'].str.contains(r'300 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '300m'
mask = athletes['EVENT'].str.contains(r'400 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'
mask = athletes['EVENT'].str.contains(r'^400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'

mask = athletes['EVENT'].str.contains(r'^400\sMeter$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m'


mask = athletes['EVENT'].str.contains(r'600 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '600m'
mask = athletes['EVENT'].str.contains(r'800 Meter Dash', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'800 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'^800m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '800m'
mask = athletes['EVENT'].str.contains(r'1500 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = athletes['EVENT'].str.contains(r'^1500m$', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m'
mask = athletes['EVENT'].str.contains(r'3000 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
mask = athletes['EVENT'].str.contains(r'3000m', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m'
mask = athletes['EVENT'].str.contains(r'5000 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = athletes['EVENT'].str.contains(r'5000m', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m'
mask = athletes['EVENT'].str.contains(r'10000 Meter Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10000m'
mask = athletes['EVENT'].str.contains(r'^10000m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'
mask = athletes['EVENT'].str.contains(r'10 Kilometer Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'


mask = athletes['EVENT'].str.contains(r'1 Mile Run', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1 mile'
mask = athletes['EVENT'].str.contains(r'Mile', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1 mile'


mask = athletes['EVENT'].str.contains(r'10\,000m', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10,000m'



# Hurdles

mask = athletes['EVENT'].str.contains(r'^80m\sHurdles$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '80m hurdles'
mask = athletes['EVENT'].str.contains(r'^80m\shurdles$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '80m hurdles'
mask = athletes['EVENT'].str.contains(r'^80\sMeter\sHurdles$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '80m hurdles'
mask = athletes['EVENT'].str.contains(r'^100m\sHurdles$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m hurdles'
mask = athletes['EVENT'].str.contains(r'^100\sMeter\sHurdles$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m hurdles'
mask = athletes['EVENT'].str.contains(r'100\sMeter\sHurdles\s\(0\.838m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m hurdles'
mask = athletes['EVENT'].str.contains(r'100m\sHurdles\s\(0\.838m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '100m hurdles'

mask = athletes['EVENT'].str.contains(r'110\sMeter\sHurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '110m hurdles'
mask = athletes['EVENT'].str.contains(r'^110m\sHurdles$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '110m hurdles'


#mask = athletes['EVENT'].str.contains(r'110\sMeter\sHurdles\s\(0\.914m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '110m hurdles'
#mask = athletes['EVENT'].str.contains(r'110m\sHurdles\s\(0\.914m\)', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = '110m hurdles'

mask = athletes['EVENT'].str.contains(r'110\sMeter\sHurdles\s\(1\.067m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '110m hurdles'
mask = athletes['EVENT'].str.contains(r'110m\sHurdles\s\(1\.067m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '110m hurdles'
mask = athletes['EVENT'].str.contains(r'110\sMeter\sHurdles\sOpen', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '110m hurdles'



mask = athletes['EVENT'].str.contains(r'^200m\sHurdles$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m hurdles'
mask = athletes['EVENT'].str.contains(r'200 Meter Hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m hurdles'
mask = athletes['EVENT'].str.contains(r'200m Hurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '200m hurdles'
mask = athletes['EVENT'].str.contains(r'200m\sHurdles\s\(0\.762m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = ' '

mask = athletes['EVENT'].str.contains(r'400m\sHurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m hurdles'
mask = athletes['EVENT'].str.contains(r'400m\sHurdles\s\(0.840m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = ' '

mask = athletes['EVENT'].str.contains(r'400\sMeter\sHurdles', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m hurdles'


mask = athletes['EVENT'].str.contains(r'400m\sHurdles\sOpen', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m hurdles'
mask = athletes['EVENT'].str.contains(r'400\sMeter\sHurdles\sOpen', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m hurdles'

mask = athletes['EVENT'].str.contains(r'400\sMeter\sHurdles\sOPEN', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m hurdles'


mask = athletes['EVENT'].str.contains(r'400\sMeter\sHurdles\s\(0\.914m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m hurdles'
mask = athletes['EVENT'].str.contains(r'400m\sHurdles\s\(0\.914m\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m hurdles'

mask = athletes['EVENT'].str.contains(r'^400\sMeter\sHurdles\s\(0\.762m\)$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m hurdles'
mask = athletes['EVENT'].str.contains(r'^400m\sHurdles\s\(0\.762m\)$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '400m hurdles'


# Throws

mask = athletes['EVENT'].str.contains(r'Javelin\sThrow\s\(600g\)', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin throw'
mask = athletes['EVENT'].str.contains(r'^Javelin\sThrow$', na=True, regex=True)  # exact match for string
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin throw'
mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s\(600g\)', na=True, regex=True)) & (athletes['GENDER']=='Female'))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin throw'
mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s600g', na=True, regex=True)) & (athletes['GENDER']=='Female'))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin throw'
mask = ((athletes['EVENT'].str.contains(r'Javelin\sThrow\s600g\)', na=True, regex=True)) & (athletes['GENDER']=='Female'))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin throw'
mask = athletes['EVENT'].str.contains(r'Javelin\sThrow\s\(800g\)', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin throw'
mask = athletes['EVENT'].str.contains(r'Javelin\sThrow\sOpen', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin throw'
mask = athletes['EVENT'].str.contains(r'Javelin\sThrow\sOPEN', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin throw'
mask = athletes['EVENT'].str.contains(r'Javelin\sThrow', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Javelin throw'



mask = athletes['EVENT'].str.contains(r'^Shot\sPut$', na=True, regex=True) # there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'

mask = (athletes['EVENT'].str.contains(r'Shot\sPut\s\(4\.00kg\)', na=True, regex=True) & (athletes['GENDER']=='Female'))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'
mask = (athletes['EVENT'].str.contains(r'Shot\sPut\s\(4kg\)', na=True, regex=True) & (athletes['GENDER']=='Female'))# there are some additional characters after Put
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'


mask = athletes['EVENT'].str.contains(r'Women\sShot\sPut\s4kg\sOpen', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'
#mask = athletes['EVENT'].str.contains(r'Men\sShot\sPut\s4kg\sOPEN', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'
mask = athletes['EVENT'].str.contains(r'Shot\sPut\sOPEN', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'


mask = athletes['EVENT'].str.contains(r'Women\sShot\sPut\s\(4kg\)', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'
mask = athletes['EVENT'].str.contains(r'Shot\sPut\s\(7\.26kg\)', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'
mask = athletes['EVENT'].str.contains(r'Shot\sPut\s7\.26kg\sOpen', na=True, regex=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'

mask = athletes['EVENT'].str.contains(r'Shot\sPut\sOpen', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'
mask = athletes['EVENT'].str.contains(r'^Shot\sput$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'


#mask = athletes['EVENT'].str.contains(r'Shot Put Masters', na=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Shot put'


mask = athletes['EVENT'].str.contains(r'^Hammer\sThrow$', na=True)  # there are some characters after Throw
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer throw'

mask = (athletes['EVENT'].str.contains(r'Hammer\sThrow\s\(4kg\)', na=True) & (athletes['GENDER']=='Female'))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer throw'
mask = athletes['EVENT'].str.contains(r'Hammer\sThrow\s\(7\.26kg\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Hammer throw'

#mask = athletes['EVENT'].str.contains(r'^Discus\sThrow    $', na=True, regex=True)
#athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus throw'
mask = athletes['EVENT'].str.contains(r'Discus\sThrow', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus throw'
mask = athletes['EVENT'].str.contains(r'Discus\sthrow', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus throw'


mask = ((athletes['EVENT'].str.contains(r'Discus\sThrow\s\(1kg\)', na=True)) & (athletes['GENDER']=='Female'))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus throw'

mask = ((athletes['EVENT'].str.contains(r'Discus\s\(1\.00kg\)', na=True))  & (athletes['GENDER']=='Female'))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus throw'



mask = athletes['EVENT'].str.contains(r'Women\sDiscus\sThrow\s\(1kg\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus throw'



mask = athletes['EVENT'].str.contains(r'Discus\sThrow\s\(2kg\)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus throw'
mask = ((athletes['EVENT'].str.contains(r'Discus\sThrow\s\(1kg\)', na=True)) & (athletes['GENDER']=='Female'))
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus throw'
mask = athletes['EVENT'].str.contains(r'Discus\sThrow\sOpen', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus throw'
mask = athletes['EVENT'].str.contains(r'Discus\sThrow\sOPEN', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Discus throw'



# Jumps

mask = athletes['EVENT'].str.contains(r'High Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High jump'

mask = athletes['EVENT'].str.contains(r'Long\sJump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Open', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long jump'
mask = athletes['EVENT'].str.contains(r'Long Jump Trial', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long jump'


mask = athletes['EVENT'].str.contains(r'Triple Jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple jump'
mask = athletes['EVENT'].str.contains(r'Pole Vault', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole vault'
mask = athletes['EVENT'].str.contains(r'High jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'High jump'
mask = athletes['EVENT'].str.contains(r'Long jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Long jump'
mask = athletes['EVENT'].str.contains(r'Triple jump', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Triple jump'
mask = athletes['EVENT'].str.contains(r'^Pole\svault$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Pole vault'

# Steeplechase

mask = athletes['EVENT'].str.contains(r'2000m S/C', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = athletes['EVENT'].str.contains(r'2000m steeplechase', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = athletes['EVENT'].str.contains(r'2000 Meter Steeplechase', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '2000m Steeplechase'
mask = athletes['EVENT'].str.contains(r'3000m S/C', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase'
mask = athletes['EVENT'].str.contains(r'3000 Meter Steeplechase', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Steeplechase'

# Marathon

mask = athletes['EVENT'].str.contains(r'Marathon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Marathon'
mask = athletes['EVENT'].str.contains(r'Half\sMarathon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Half marathon'

# Walk

mask = athletes['EVENT'].str.contains(r'1500m Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Racewalk'
mask = athletes['EVENT'].str.contains(r'1500 Meter Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '1500m Racewalk'
mask = athletes['EVENT'].str.contains(r'3000m Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Racewalk'
mask = athletes['EVENT'].str.contains(r'3000 Meter Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '3000m Racewalk'
mask = athletes['EVENT'].str.contains(r'5000 Meter Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Racewalk'
mask = athletes['EVENT'].str.contains(r'5000m Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '5000m Racewalk'
mask = athletes['EVENT'].str.contains(r'10000 Meter Race Walk', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '10000m Racewalk'

# Relay

mask = athletes['EVENT'].str.contains(r'4x80m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 80m relay'
mask = athletes['EVENT'].str.contains(r'^4\sx\s100m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m relay'
mask = athletes['EVENT'].str.contains(r'4x100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m relay'
mask = athletes['EVENT'].str.contains(r'4 X 100m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m relay'
mask = athletes['EVENT'].str.contains(r'4x400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m relay'
mask = athletes['EVENT'].str.contains(r'4 X 400m Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m relay'
mask = athletes['EVENT'].str.contains(r'4x100 Meter Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 100m relay'
mask = athletes['EVENT'].str.contains(r'4x400 Meter Relay', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m relay'
mask = athletes['EVENT'].str.contains(r'^4\sx\s400m$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = '4 x 400m relay'

# Decathlon/Heptathlon

mask = athletes['EVENT'].str.contains(r'^Heptathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'^Decathlon$', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'
mask = athletes['EVENT'].str.contains(r'Heptathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Heptathlon'
mask = athletes['EVENT'].str.contains(r'Decathlon', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Decathlon'

# Cross Country

mask = athletes['EVENT'].str.contains(r'10 Kilometer Run (Cross Country)', na=True)
athletes.loc[mask, 'MAPPED_EVENT'] = 'Cross Country'



/var/folders/q5/yf8g5p896_b94gkbhqcjx3t40000gn/T/ipykernel_60343/2949763865.py:340: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = athletes['EVENT'].str.contains(r'10 Kilometer Run (Cross Country)', na=True)


In [160]:
athletes

,Date,EVENT,Name,Age,Team,Result,Wind m/s,Competition,Year D.O.B.,"Info, if any",REGION,SOURCE,date_extract,YEAR,GENDER,COMPETITION,STAGE,DIVISION,REMARKS,MAPPED_EVENT
0,2024-12-19,Men 800 Meter Run,"THANA RAJAN, THIRUBEN",24,SINGAPORE,1:53.51,-,International - Strive (Australia),2000,-,International,SAA Database,19-12,2024,Male,Strive (Australia),NaN,NaN,NaN,800m
1,2024-12-19,Men 1500 Meter Run,"LIM, OLIVER",24,SINGAPORE,4:01.49,-,International - Vic Milers (Australia),1999,-,International,SAA Database,19-12,2024,Male,Vic Milers (Australia),NaN,NaN,NaN,1500m
2,2024-12-07,Men 60 Meter Dash,"TAN, DARYL",23,SINGAPORE,6.84,1.4,"International - Strive Program B, Perth (Austr...",2001,-,International,SAA Database,07-12,2024,Male,"Strive Program B, Perth (Australia)",NaN,NaN,NaN,60m
3,2024-12-07,Men 200 Meter Dash,"TAN, DARYL",23,SINGAPORE,21.68,1.2,"International - Strive Program B, Perth (Austr...",2001,-,International,SAA Database,07-12,2024,Male,"Strive Program B, Perth (Australia)",NaN,NaN,NaN,200m
4,2024-12-07,Men 60 Meter Dash,"TOH JUN XI, TEDD",22,SINGAPORE,6.97,1.3,"International - Strive Program B, Perth (Austr...",2002,-,International,SAA Database,07-12,2024,Male,"Strive Program B, Perth (Australia)",NaN,NaN,NaN,60m
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6535,2011-04-16,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,14.91,-,International - Australia Athletics Cships,1990,"- Preliminaries Olympic Park, Melbourne",International,SAA Database,16-04,2011,Male,Australia Athletics Cships,Prelim,NaN,NaN,Shot put
6536,2011-03-27,Women 10000 Meter Run,Renuka Satianathan,24,Singapore,37:15.87,-,International - Queensland Open & AWD Cships,1987,"- Finals State Athletics Facility, Nathan Quee...",International,SAA Database,27-03,2011,Female,Queensland Open & AWD Cships,Final,NaN,NaN,10000m
6537,2011-03-27,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,15.32,-,International - Athletic Victoria Throwers Mee...,1990,"- Finals Proclamations Park, Ringwood, Melbourne",International,SAA Database,27-03,2011,Male,Athletic Victoria Throwers Meet #6,Final,NaN,NaN,Shot put
6538,2011-03-11,Men 5000 Meter Race Walk,Peter James Back,-,Singapore,26:53,HT,International - Thai Veterans Athletics Cships,-,"Open Final Bangkok, Thailand",International,SAA Database,11-03,2011,Male,Thai Veterans Athletics Cships,Final,NaN,NaN,5000m Racewalk


In [161]:
# Extract ranking

def rank(string):
    
    

    try:
        
        slash = re.search(r'\/', string) 
        
        d1 = re.search(r'^\dpos', string)
        d2 = re.search(r'^\d\dpos', string)
        d3 = re.search(r'^\d\d\d\dpos', string)
        d4 = re.search(r'^\d\d\dpos', string)
        
        print(d1, d2, d3, d4)
        
        print(slash)
        
        if slash:
     
            start = slash.start()-2
    
            end = slash.start()
    
            pos = string[start:end]
        
        elif d1:
            
            pos = string[:1]
            
            print(d1)
            
        elif d2:
            
            pos = string[:2]
            
            print(d2)
            
        elif d3:
            
            pos = string[:4]
            
        elif d4:
            
            pos = string[:3]
            
            
        else:
            
            pos=''
            
    except:
        
        pos = ''
    
    return pos
    

athletes['RANK'] = athletes['Info, if any'].apply(rank)


None None None None
None
None None None None
None
None None None None
None
None None None None
None
None None None None
None
None None None None
None
None None None None
<re.Match object; span=(8, 9), match='/'>
None None None None
<re.Match object; span=(12, 13), match='/'>
None None None None
<re.Match object; span=(12, 13), match='/'>
None None None None
<re.Match object; span=(8, 9), match='/'>
None None None None
<re.Match object; span=(12, 13), match='/'>
None None None None
<re.Match object; span=(8, 9), match='/'>
None None None None
<re.Match object; span=(8, 9), match='/'>
None None None None
<re.Match object; span=(8, 9), match='/'>
None None None None
<re.Match object; span=(8, 9), match='/'>
None None None None
<re.Match object; span=(8, 9), match='/'>
None None None None
<re.Match object; span=(8, 9), match='/'>
None None None None
<re.Match object; span=(8, 9), match='/'>
None None None None
<re.Match object; span=(8, 9), match='/'>
None None None None
<re.Match object; 

In [162]:
# Extract date

def date(string):
    
    
    try:
        
        slash=re.search(r'2025', string)
        
        end = slash.start()
    
        date = string[:end]
            
    except:
        
        date = ''
    
    return date
    

#athletes['DATE'] = athletes['Date'].apply(date)


In [163]:
athletes

,Date,EVENT,Name,Age,Team,Result,Wind m/s,Competition,Year D.O.B.,"Info, if any",...,SOURCE,date_extract,YEAR,GENDER,COMPETITION,STAGE,DIVISION,REMARKS,MAPPED_EVENT,RANK
0,2024-12-19,Men 800 Meter Run,"THANA RAJAN, THIRUBEN",24,SINGAPORE,1:53.51,-,International - Strive (Australia),2000,-,...,SAA Database,19-12,2024,Male,Strive (Australia),NaN,NaN,NaN,800m,
1,2024-12-19,Men 1500 Meter Run,"LIM, OLIVER",24,SINGAPORE,4:01.49,-,International - Vic Milers (Australia),1999,-,...,SAA Database,19-12,2024,Male,Vic Milers (Australia),NaN,NaN,NaN,1500m,
2,2024-12-07,Men 60 Meter Dash,"TAN, DARYL",23,SINGAPORE,6.84,1.4,"International - Strive Program B, Perth (Austr...",2001,-,...,SAA Database,07-12,2024,Male,"Strive Program B, Perth (Australia)",NaN,NaN,NaN,60m,
3,2024-12-07,Men 200 Meter Dash,"TAN, DARYL",23,SINGAPORE,21.68,1.2,"International - Strive Program B, Perth (Austr...",2001,-,...,SAA Database,07-12,2024,Male,"Strive Program B, Perth (Australia)",NaN,NaN,NaN,200m,
4,2024-12-07,Men 60 Meter Dash,"TOH JUN XI, TEDD",22,SINGAPORE,6.97,1.3,"International - Strive Program B, Perth (Austr...",2002,-,...,SAA Database,07-12,2024,Male,"Strive Program B, Perth (Australia)",NaN,NaN,NaN,60m,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6535,2011-04-16,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,14.91,-,International - Australia Athletics Cships,1990,"- Preliminaries Olympic Park, Melbourne",...,SAA Database,16-04,2011,Male,Australia Athletics Cships,Prelim,NaN,NaN,Shot put,
6536,2011-03-27,Women 10000 Meter Run,Renuka Satianathan,24,Singapore,37:15.87,-,International - Queensland Open & AWD Cships,1987,"- Finals State Athletics Facility, Nathan Quee...",...,SAA Database,27-03,2011,Female,Queensland Open & AWD Cships,Final,NaN,NaN,10000m,
6537,2011-03-27,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,15.32,-,International - Athletic Victoria Throwers Mee...,1990,"- Finals Proclamations Park, Ringwood, Melbourne",...,SAA Database,27-03,2011,Male,Athletic Victoria Throwers Meet #6,Final,NaN,NaN,Shot put,
6538,2011-03-11,Men 5000 Meter Race Walk,Peter James Back,-,Singapore,26:53,HT,International - Thai Veterans Athletics Cships,-,"Open Final Bangkok, Thailand",...,SAA Database,11-03,2011,Male,Thai Veterans Athletics Cships,Final,NaN,NaN,5000m Racewalk,


In [164]:
# Extract heat

def heat(string):
    
    
    try:
        
        heat=re.findall(r'h\d|H\d|h\d\d|H\d\d', string)
        
        heat1=heat[0]
                
            
    except:
        
        heat1 = ''
    
    return heat1
    

athletes['HEAT'] = athletes['Info, if any'].apply(heat)


In [165]:
athletes

,Date,EVENT,Name,Age,Team,Result,Wind m/s,Competition,Year D.O.B.,"Info, if any",...,date_extract,YEAR,GENDER,COMPETITION,STAGE,DIVISION,REMARKS,MAPPED_EVENT,RANK,HEAT
0,2024-12-19,Men 800 Meter Run,"THANA RAJAN, THIRUBEN",24,SINGAPORE,1:53.51,-,International - Strive (Australia),2000,-,...,19-12,2024,Male,Strive (Australia),NaN,NaN,NaN,800m,,
1,2024-12-19,Men 1500 Meter Run,"LIM, OLIVER",24,SINGAPORE,4:01.49,-,International - Vic Milers (Australia),1999,-,...,19-12,2024,Male,Vic Milers (Australia),NaN,NaN,NaN,1500m,,
2,2024-12-07,Men 60 Meter Dash,"TAN, DARYL",23,SINGAPORE,6.84,1.4,"International - Strive Program B, Perth (Austr...",2001,-,...,07-12,2024,Male,"Strive Program B, Perth (Australia)",NaN,NaN,NaN,60m,,
3,2024-12-07,Men 200 Meter Dash,"TAN, DARYL",23,SINGAPORE,21.68,1.2,"International - Strive Program B, Perth (Austr...",2001,-,...,07-12,2024,Male,"Strive Program B, Perth (Australia)",NaN,NaN,NaN,200m,,
4,2024-12-07,Men 60 Meter Dash,"TOH JUN XI, TEDD",22,SINGAPORE,6.97,1.3,"International - Strive Program B, Perth (Austr...",2002,-,...,07-12,2024,Male,"Strive Program B, Perth (Australia)",NaN,NaN,NaN,60m,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6535,2011-04-16,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,14.91,-,International - Australia Athletics Cships,1990,"- Preliminaries Olympic Park, Melbourne",...,16-04,2011,Male,Australia Athletics Cships,Prelim,NaN,NaN,Shot put,,
6536,2011-03-27,Women 10000 Meter Run,Renuka Satianathan,24,Singapore,37:15.87,-,International - Queensland Open & AWD Cships,1987,"- Finals State Athletics Facility, Nathan Quee...",...,27-03,2011,Female,Queensland Open & AWD Cships,Final,NaN,NaN,10000m,,
6537,2011-03-27,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,15.32,-,International - Athletic Victoria Throwers Mee...,1990,"- Finals Proclamations Park, Ringwood, Melbourne",...,27-03,2011,Male,Athletic Victoria Throwers Meet #6,Final,NaN,NaN,Shot put,,
6538,2011-03-11,Men 5000 Meter Race Walk,Peter James Back,-,Singapore,26:53,HT,International - Thai Veterans Athletics Cships,-,"Open Final Bangkok, Thailand",...,11-03,2011,Male,Thai Veterans Athletics Cships,Final,NaN,NaN,5000m Racewalk,,


In [166]:
# Extract qualitication

def qual(string):
    
    
    try:
        
        qual=re.findall(r'\sq|\sQ', string)
        qual1=qual[0]
        
                
            
    except:
        
        qual1 = ''
    
    return qual1
    

athletes['QUALIFICATION'] = athletes['Info, if any'].apply(qual)


In [167]:
# Extract event class

def event_class(string):
    
    
    try:
        
        left=re.search(r'\(', string)
        right=re.search(r'\)', string)
        
        
        type = string[left.start()+1:right.start()]
        
                 
    except:
        
        type = ''
    
    return type
    

athletes['EVENT_CLASS'] = athletes['EVENT'].apply(event_class)


In [168]:
athletes

,Date,EVENT,Name,Age,Team,Result,Wind m/s,Competition,Year D.O.B.,"Info, if any",...,GENDER,COMPETITION,STAGE,DIVISION,REMARKS,MAPPED_EVENT,RANK,HEAT,QUALIFICATION,EVENT_CLASS
0,2024-12-19,Men 800 Meter Run,"THANA RAJAN, THIRUBEN",24,SINGAPORE,1:53.51,-,International - Strive (Australia),2000,-,...,Male,Strive (Australia),NaN,NaN,NaN,800m,,,,
1,2024-12-19,Men 1500 Meter Run,"LIM, OLIVER",24,SINGAPORE,4:01.49,-,International - Vic Milers (Australia),1999,-,...,Male,Vic Milers (Australia),NaN,NaN,NaN,1500m,,,,
2,2024-12-07,Men 60 Meter Dash,"TAN, DARYL",23,SINGAPORE,6.84,1.4,"International - Strive Program B, Perth (Austr...",2001,-,...,Male,"Strive Program B, Perth (Australia)",NaN,NaN,NaN,60m,,,,
3,2024-12-07,Men 200 Meter Dash,"TAN, DARYL",23,SINGAPORE,21.68,1.2,"International - Strive Program B, Perth (Austr...",2001,-,...,Male,"Strive Program B, Perth (Australia)",NaN,NaN,NaN,200m,,,,
4,2024-12-07,Men 60 Meter Dash,"TOH JUN XI, TEDD",22,SINGAPORE,6.97,1.3,"International - Strive Program B, Perth (Austr...",2002,-,...,Male,"Strive Program B, Perth (Australia)",NaN,NaN,NaN,60m,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6535,2011-04-16,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,14.91,-,International - Australia Athletics Cships,1990,"- Preliminaries Olympic Park, Melbourne",...,Male,Australia Athletics Cships,Prelim,NaN,NaN,Shot put,,,,7.26kg
6536,2011-03-27,Women 10000 Meter Run,Renuka Satianathan,24,Singapore,37:15.87,-,International - Queensland Open & AWD Cships,1987,"- Finals State Athletics Facility, Nathan Quee...",...,Female,Queensland Open & AWD Cships,Final,NaN,NaN,10000m,,,Q,
6537,2011-03-27,Men Shot Put (7.26kg),Scott Wong Wei Gen,21,Singapore,15.32,-,International - Athletic Victoria Throwers Mee...,1990,"- Finals Proclamations Park, Ringwood, Melbourne",...,Male,Athletic Victoria Throwers Meet #6,Final,NaN,NaN,Shot put,,,,7.26kg
6538,2011-03-11,Men 5000 Meter Race Walk,Peter James Back,-,Singapore,26:53,HT,International - Thai Veterans Athletics Cships,-,"Open Final Bangkok, Thailand",...,Male,Thai Veterans Athletics Cships,Final,NaN,NaN,5000m Racewalk,,,,


In [169]:
athletes.to_csv("SAA_external_database.csv", encoding="utf-8")

In [170]:
athletes = athletes.rename(columns={'Info, if any': 'REMARKS', 'Wind m/s': "WIND", 'Age': "AGE", 'Result': "RESULT", 'date_extract': "DATE", 'Year D.O.B.': "DOB"})

In [171]:
athletes = athletes.drop(['Date', 'Competition', 'EVENT'], axis=1)

In [172]:
athletes = athletes.rename(columns={'MAPPED_EVENT': "EVENT", 'Name': "NAME", 'Team': "TEAM"})

In [173]:
athletes = athletes[athletes['YEAR']=='2023']

In [174]:
athletes

,NAME,AGE,TEAM,RESULT,WIND,DOB,REMARKS,REGION,SOURCE,DATE,...,GENDER,COMPETITION,STAGE,DIVISION,REMARKS,EVENT,RANK,HEAT,QUALIFICATION,EVENT_CLASS
530,"NORHISHAM, JAMIE EL-REDHA ANG",19,Singapore Sports School,11.01,-0.2,2004,Heats 1/8pos q,International,SAA Database,20-12,...,Male,SAECA-BJSS Athletics Championship,Heats,NaN,NaN,100m,1,,q,
531,"NORHISHAM, JAMIE EL-REDHA ANG",19,Singapore Sports School,11.08,-0.1,2004,Final 8/8pos,International,SAA Database,20-12,...,Male,SAECA-BJSS Athletics Championship,Final,NaN,NaN,100m,8,,,
532,"MUTHUKUMARAN, RAAM KUMAR",19,Singapore Sports School,3:20.75,-,2004,Final 1/8pos,International,SAA Database,20-12,...,Male,SAECA-BJSS Athletics Championship,Final,NaN,NaN,4 x 400m relay,1,,,
533,"NORHISHAM, JAMIE EL-REDHA ANG",19,Singapore Sports School,22.32,-0.3,2004,Final 3/8pos,International,SAA Database,19-12,...,Male,SAECA-BJSS Athletics Championship,Final,NaN,NaN,200m,3,,,
534,"YUSRAN, NIQ RIFDI RAFIQI",16,Singapore Sports School,55.05,-,2007,Heats 4/8pos,International,SAA Database,19-12,...,Male,SAECA-BJSS Athletics Championship,Heats,NaN,NaN,400m,4,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2114,"ROZARIO, TIA LOUISE",23,PRINCETON,5.52,Indoor,2000,"5/8pos, sanctioned",International,SAA Database,28-01,...,Female,Harvard-Yale-Princeton,NaN,NaN,NaN,Long jump,,,,
2115,"KAM, KAMPTON",22,INDIVIDUAL,2.08,-,2001,"Sanctioned,1pos",International,SAA Database,21-01,...,Male,Wesley A Brown Invitational,NaN,NaN,NaN,High jump,,,,
2116,"PHUA, JASMIN",22,Individual,42.83,-,2000,-,International,SAA Database,20-01,...,Female,Strive Season Program B,NaN,NaN,NaN,Discus throw,,,,1kg
2117,"KAM, KAMPTON",22,Individual,2.06,-,2000,"Sanctioned,1pos",International,SAA Database,14-01,...,Male,Penn 10-Team Select,NaN,NaN,NaN,High jump,,,,


In [175]:
athletes.to_csv("athletes_2023.csv", encoding='utf-8')

In [176]:
# Map general event category 

athletes.loc[athletes['EVENT'].str.contains(r'100m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
athletes.loc[athletes['EVENT'].str.contains(r'400m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
athletes.loc[athletes['EVENT'].str.contains(r'600m', na=True), 'CATEGORY_EVENT'] = 'Mid'
athletes.loc[athletes['EVENT'].str.contains(r'60m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
athletes.loc[athletes['EVENT'].str.contains(r'200m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
athletes.loc[athletes['EVENT'].str.contains(r'50m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
athletes.loc[athletes['EVENT'].str.contains(r'80m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
athletes.loc[athletes['EVENT'].str.contains(r'300m', na=True), 'CATEGORY_EVENT'] = 'Sprint'
athletes.loc[athletes['EVENT'].str.contains(r'3000m', na=True), 'CATEGORY_EVENT'] = 'Long'
athletes.loc[athletes['EVENT'].str.contains(r'Mile', na=True), 'CATEGORY_EVENT'] = 'Mid'
athletes.loc[athletes['EVENT'].str.contains(r'mile', na=True), 'CATEGORY_EVENT'] = 'Mid'

athletes.loc[athletes['EVENT'].str.contains(r'5000m', na=True), 'CATEGORY_EVENT'] = 'Long'
athletes.loc[athletes['EVENT'].str.contains(r'2000m', na=True), 'CATEGORY_EVENT'] = 'Mid'
athletes.loc[athletes['EVENT'].str.contains(r'800m', na=True), 'CATEGORY_EVENT'] = 'Mid'
athletes.loc[athletes['EVENT'].str.contains(r'1500m', na=True), 'CATEGORY_EVENT'] = 'Mid'
athletes.loc[athletes['EVENT'].str.contains(r'10,000m', na=True), 'CATEGORY_EVENT'] = 'Long'
athletes.loc[athletes['EVENT'].str.contains(r'10000m', na=True), 'CATEGORY_EVENT'] = 'Long'
athletes.loc[athletes['EVENT'].str.contains(r'5km', na=True), 'CATEGORY_EVENT'] = 'Long'
athletes.loc[athletes['EVENT'].str.contains(r'10km', na=True), 'CATEGORY_EVENT'] = 'Long'
athletes.loc[athletes['EVENT'].str.contains(r'Cross Country', na=True), 'CATEGORY_EVENT'] = 'Cross Country'


# Override values

athletes.loc[athletes['EVENT'].str.contains(r'Discus', na=True), 'CATEGORY_EVENT'] = 'Throw'
athletes.loc[athletes['EVENT'].str.contains(r'Javelin', na=True), 'CATEGORY_EVENT'] = 'Throw'
athletes.loc[athletes['EVENT'].str.contains(r'Shot', na=True), 'CATEGORY_EVENT'] = 'Throw'
athletes.loc[athletes['EVENT'].str.contains(r'Vault', na=True), 'CATEGORY_EVENT'] = 'Jump'
athletes.loc[athletes['EVENT'].str.contains(r'vault', na=True), 'CATEGORY_EVENT'] = 'Jump'

athletes.loc[athletes['EVENT'].str.contains(r'Throw', na=True), 'CATEGORY_EVENT'] = 'Throw'
athletes.loc[athletes['EVENT'].str.contains(r'Jump', na=True), 'CATEGORY_EVENT'] = 'Jump'
athletes.loc[athletes['EVENT'].str.contains(r'jump', na=True), 'CATEGORY_EVENT'] = 'Jump'

athletes.loc[athletes['EVENT'].str.contains(r'Relay', na=True), 'CATEGORY_EVENT'] = 'Relay'
athletes.loc[athletes['EVENT'].str.contains(r'steeple', na=True), 'CATEGORY_EVENT'] = 'Steeple'
athletes.loc[athletes['EVENT'].str.contains(r'Walk', na=True), 'CATEGORY_EVENT'] = 'Walk'
athletes.loc[athletes['EVENT'].str.contains(r'Hurdles', na=True), 'CATEGORY_EVENT'] = 'Hurdles'
athletes.loc[athletes['EVENT'].str.contains(r'hurdles', na=True), 'CATEGORY_EVENT'] = 'Hurdles'

athletes.loc[athletes['EVENT'].str.contains(r'Pentathlon', na=True), 'CATEGORY_EVENT'] = 'Pentathlon'
athletes.loc[athletes['EVENT'].str.contains(r'Triathlon', na=True), 'CATEGORY_EVENT'] = 'Triathlon'
athletes.loc[athletes['EVENT'].str.contains(r'Decathlon', na=True), 'CATEGORY_EVENT'] = 'Decathlon'
athletes.loc[athletes['EVENT'].str.contains(r'Marathon', na=True), 'CATEGORY_EVENT'] = 'Marathon'
athletes.loc[athletes['EVENT'].str.contains(r'marathon', na=True), 'CATEGORY_EVENT'] = 'Marathon'
athletes.loc[athletes['EVENT'].str.contains(r'4 x 100m', na=True), 'CATEGORY_EVENT'] = 'Relay'
athletes.loc[athletes['EVENT'].str.contains(r'4 x 400m', na=True), 'CATEGORY_EVENT'] = 'Relay'
athletes.loc[athletes['EVENT'].str.contains(r'Steeplechase', na=True), 'CATEGORY_EVENT'] = 'Steeple'




In [177]:
# Extract date

def time(string):
    
    
    try:
        
        slash=re.search(r'AM|PM', string)
        
        end = slash.start()
    
        time = string[:end]
            
    except:
        
        time = ''
    
    return time
    

#athletes['RESULT_CLEANED'] = athletes['RESULT'].apply(time)


In [180]:
athletes.columns

Index(['NAME', 'AGE', 'TEAM', 'RESULT', 'WIND', 'DOB', 'REMARKS', 'REGION',
       'SOURCE', 'DATE', 'YEAR', 'GENDER', 'COMPETITION', 'STAGE', 'DIVISION',
       'REMARKS', 'EVENT', 'RANK', 'HEAT', 'QUALIFICATION', 'EVENT_CLASS',
       'CATEGORY_EVENT', 'LAST_NAME', 'FIRST_NAME', 'OTHER_NAME', 'SEED',
       'LANE', 'ATHLETE_ID', 'TIMESTAMP', 'TAG_ID', 'POINTS', 'GROUP',
       'SESSION', 'DISTANCE', 'HOST_CITY', 'SUB_EVENT', 'DICT_RESULTS',
       'RX_TIME'],
      dtype='object')

In [179]:
# Convert to NEW SCHEMA

athletes['LAST_NAME'] = ''
athletes['FIRST_NAME'] = ''
athletes['OTHER_NAME'] = ''
athletes['SEED'] = ''
athletes['LANE'] = ''
athletes['ATHLETE_ID'] = ''
athletes['TIMESTAMP'] = ''
athletes['TAG_ID'] = ''
athletes['POINTS'] = ''
athletes['GROUP'] = ''
athletes['SESSION']=''
athletes['DISTANCE']=''
athletes['POINTS']=''
athletes['HOST_CITY']=''
athletes['SUB_EVENT']=''
athletes['DICT_RESULTS']=''
athletes['RX_TIME']=''






athletes = athletes.reindex(columns= ['FIRST_NAME', 'LAST_NAME', 'OTHER_NAME', 'NAME', 'RANK', 'TAG_ID', 'TEAM', 'SEED', 'RESULT', 'QUALIFICATION',
                                        'HEAT', 'LANE', 'WIND', 'EVENT', 'DIVISION', 'STAGE', 'POINTS', 'AGE', 'GENDER', 'UNIQUE_ID', 'NATIONALITY',
                                        'DICT_RESULTS', 'YEAR', 'DATE', 'COMPETITION', 'REGION', 'DOB', 'GROUP', 'CATEGORY_EVENT', 'ATHLETE_ID',
                                        'SOURCE', 'REMARKS', 'TIMESTAMP', 'VENUE', 'SUB_EVENT', 'SESSION', 'EVENT_CLASS', 'DISTANCE', 'HOST_CITY', 'RX_TIME'])


ValueError: cannot reindex on an axis with duplicate labels

In [55]:
athletes.to_csv("external_data_processed_file.csv", encoding='utf-8')

In [56]:
athletes

,FIRST_NAME,LAST_NAME,OTHER_NAME,NAME,RANK,TAG_ID,TEAM,SEED,RESULT,QUALIFICATION,...,SOURCE,REMARKS,TIMESTAMP,VENUE,SUB_EVENT,SESSION,EVENT_CLASS,DISTANCE,HOST_CITY,RX_TIME
0,,,,NaN,,,NaN,,1:53.51,,...,SAA Database,NaN,,NaN,,,,,,
1,,,,NaN,,,NaN,,4:01.49,,...,SAA Database,NaN,,NaN,,,,,,
2,,,,NaN,,,NaN,,6.84,,...,SAA Database,NaN,,NaN,,,,,,
3,,,,NaN,,,NaN,,21.68,,...,SAA Database,NaN,,NaN,,,,,,
4,,,,NaN,,,NaN,,6.97,,...,SAA Database,NaN,,NaN,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6535,,,,NaN,,,NaN,,14.91,,...,SAA Database,NaN,,NaN,,,7.26kg,,,
6536,,,,NaN,,,NaN,,37:15.87,Q,...,SAA Database,NaN,,NaN,,,,,,
6537,,,,NaN,,,NaN,,15.32,,...,SAA Database,NaN,,NaN,,,7.26kg,,,
6538,,,,NaN,,,NaN,,26:53,,...,SAA Database,NaN,,NaN,,,,,,
